# Projeto: Fundamentos de Machine Learning

**Contexto:** A empresa Data Money fornece serviços de consultoria de Análise e Ciência de Dados para grandes empresas no Brasil e no exterior.

O seu principal diferencial de mercado em relação aos concorrentes é o alto retorno financeiro para as empresas clientes, graças a performance de seus algoritmos de Machine Learning.

A Data Money acredita que a expertise no treinamento e ajuste fino dos algoritmos, feito pelos Cientistas de Dados da empresa, é a principal motivo dos ótimos resultados que as consultorias vem entregando aos seus clientes.

Para continuar crescendo a expertise do time, os Cientistas de Dados acreditam que é extremamente importante realizar ensaios nos algoritmos de Machine Learning para adquirir uma experiência cada vez maior sobre o seu funcionamento e em quais cenários as performances são máximas e mínimas, para que a escolha do algoritmo para cada situação seja a mais correta possível.

**Objetivo:** Como Cientista de Dados recém contratado pela empresa, a sua principal tarefa será realizar 3 ensaios com algoritmos de Classificação, Regressão e Clusterização, a fim de extrair aprendizados sobre o seu funcionamento em determinados cenário e conseguir transmitir esse conhecimento para o restante do time.

## 0.0 Importação das bibliotecas

In [1]:
# Bibliotecas gerais
import numpy as np
import pandas as pd

# Classificação
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Verificação de desbalanceamento das classes
def razao_classes(n_classe0, n_classe1):
  if n_classe0 > n_classe1:
    maior = n_classe0
    menor = n_classe1
  elif n_classe1 > n_classe0:
    maior = n_classe1
    menor = n_classe0
  else:
    maior, menor = n_classe0, n_classe1

  razao_classe = menor / maior

  if razao_classe > 10:
    print('Desbalanceado')
  else:
    print('Balanceado')

  return


# Regressão
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, root_mean_squared_error, mean_squared_error, mean_absolute_error

# Cálculo do MAPE (função customizada)
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Agrupamento (Clustering)
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.metrics import silhouette_score

## 1.0 Ensaio de Classificação

### 1.1 Importando e lendo o dataset

In [33]:
# Treino
X_treino = pd.read_csv('Ensaio_de_Classificacao/X_training.csv')
y_treino = pd.read_csv('Ensaio_de_Classificacao/y_training.csv')

# Validação
X_val = pd.read_csv('Ensaio_de_Classificacao/X_validation.csv')
y_val = pd.read_csv('Ensaio_de_Classificacao/y_validation.csv')

# Teste
X_teste = pd.read_csv('Ensaio_de_Classificacao/X_test.csv')
y_teste = pd.read_csv('Ensaio_de_Classificacao/y_test.csv')

# Corrigindo os rótulos
y_treino_1d = y_treino.to_numpy().ravel()
y_val_1d = y_val.to_numpy().ravel()
y_teste_1d = y_teste.to_numpy().ravel()

# Treino + Validação
X_concat_treino_val = pd.concat([X_treino, X_val], axis=0)
y_concat_treino_val = np.concatenate([y_treino_1d, y_val_1d])


### 1.2 Explorando o dataset

#### 1.2.1 Treino

In [34]:
X_treino.head()

,id,customer_type,age,class,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,food_and_drink,...,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,gender_Female,gender_Male,type_of_travel_business_travel,type_of_travel_personal_travel
0,13508,1,0.500000,0.0,0.039580,0.6,0.6,0.6,0.6,1.0,...,0.5,1.00,0.6,0.4,0.000000,0.013848,1.0,0.0,1.0,0.0
1,28874,1,0.243590,0.0,0.205775,0.6,0.4,0.4,0.4,0.6,...,0.5,0.50,0.2,0.6,0.000000,0.000000,0.0,1.0,1.0,0.0
2,21484,0,0.435897,1.0,0.026858,0.6,0.6,0.6,0.2,1.0,...,0.0,1.00,0.6,1.0,0.000000,0.000000,1.0,0.0,1.0,0.0
3,48280,1,0.589744,0.5,0.041397,0.6,1.0,0.6,0.6,0.8,...,0.0,1.00,0.4,0.4,0.029499,0.020772,1.0,0.0,0.0,1.0
4,472,0,0.423077,1.0,0.016559,0.2,0.2,0.2,0.8,0.6,...,1.0,0.75,0.8,0.6,0.021632,0.019782,0.0,1.0,1.0,0.0


In [35]:
X_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72515 entries, 0 to 72514
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 72515 non-null  int64  
 1   customer_type                      72515 non-null  int64  
 2   age                                72515 non-null  float64
 3   class                              72515 non-null  float64
 4   flight_distance                    72515 non-null  float64
 5   inflight_wifi_service              72515 non-null  float64
 6   departure_arrival_time_convenient  72515 non-null  float64
 7   ease_of_online_booking             72515 non-null  float64
 8   gate_location                      72515 non-null  float64
 9   food_and_drink                     72515 non-null  float64
 10  online_boarding                    72515 non-null  float64
 11  seat_comfort                       72515 non-null  flo

In [36]:
y_treino.head()

,0
0,1
1,1
2,0
3,0
4,0


In [37]:
y_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72515 entries, 0 to 72514
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       72515 non-null  int64
dtypes: int64(1)
memory usage: 566.7 KB


In [38]:
y_treino_count = y_treino.value_counts()
y_treino_count

,count
0,
0,41087
1,31428


In [39]:
razao_classes(y_treino_count[0], y_treino_count[1])

Balanceado


#### 1.2.2 Validação

In [40]:
X_val.head()

,id,customer_type,age,class,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,food_and_drink,...,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,gender_Female,gender_Male,type_of_travel_business_travel,type_of_travel_personal_travel
0,75635,1,0.525641,1.0,0.714055,1.0,1.0,1.0,1.0,0.8,...,0.75,0.75,0.8,0.8,0.000000,0.000000,0.0,1.0,1.0,0.0
1,106136,1,0.615385,1.0,0.054725,0.4,1.0,1.0,1.0,0.4,...,0.25,0.00,0.4,0.6,0.004916,0.004946,0.0,1.0,1.0,0.0
2,94469,1,0.435897,1.0,0.342690,0.0,0.2,0.2,0.6,0.8,...,0.25,0.75,0.4,0.6,0.004916,0.000000,1.0,0.0,1.0,0.0
3,42104,1,0.615385,1.0,0.649838,1.0,1.0,0.6,1.0,1.0,...,0.75,0.75,0.8,0.8,0.000000,0.000989,1.0,0.0,1.0,0.0
4,78762,1,0.282051,0.0,0.100363,0.4,0.6,0.6,0.6,0.4,...,0.75,0.75,0.6,0.4,0.093412,0.090010,0.0,1.0,1.0,0.0


In [41]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31079 entries, 0 to 31078
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 31079 non-null  int64  
 1   customer_type                      31079 non-null  int64  
 2   age                                31079 non-null  float64
 3   class                              31079 non-null  float64
 4   flight_distance                    31079 non-null  float64
 5   inflight_wifi_service              31079 non-null  float64
 6   departure_arrival_time_convenient  31079 non-null  float64
 7   ease_of_online_booking             31079 non-null  float64
 8   gate_location                      31079 non-null  float64
 9   food_and_drink                     31079 non-null  float64
 10  online_boarding                    31079 non-null  float64
 11  seat_comfort                       31079 non-null  flo

In [42]:
y_val.head()

,0
0,1
1,0
2,1
3,1
4,0


In [43]:
y_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31079 entries, 0 to 31078
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       31079 non-null  int64
dtypes: int64(1)
memory usage: 242.9 KB


In [44]:
y_val_count = y_val.value_counts()
y_val_count

,count
0,
0,17610
1,13469


In [45]:
razao_classes(y_val_count[0], y_val_count[1])

Balanceado


#### 1.2.3 Teste

In [46]:
X_teste.head()

,id,customer_type,age,class,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,food_and_drink,...,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,gender_Female,gender_Male,type_of_travel_business_travel,type_of_travel_personal_travel
0,19556,1,0.576923,0.0,0.026050,1.0,0.8,0.6,0.8,0.6,...,1.00,0.25,1.0,1.0,0.049164,0.043521,1.0,0.0,1.0,0.0
1,90035,1,0.371795,1.0,0.571890,0.2,0.2,0.6,0.2,1.0,...,0.75,0.50,0.8,1.0,0.000000,0.000000,1.0,0.0,1.0,0.0
2,12360,0,0.166667,0.0,0.032512,0.4,0.0,0.4,0.8,0.4,...,0.50,0.25,0.4,0.4,0.000000,0.000000,0.0,1.0,1.0,0.0
3,77959,1,0.474359,1.0,0.675687,0.0,0.0,0.0,0.4,0.6,...,0.00,0.50,0.2,0.8,0.000000,0.005935,0.0,1.0,1.0,0.0
4,36875,1,0.538462,0.0,0.232431,0.4,0.6,0.8,0.6,0.8,...,0.25,0.75,0.4,0.8,0.000000,0.019782,1.0,0.0,1.0,0.0


In [47]:
X_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25893 entries, 0 to 25892
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 25893 non-null  int64  
 1   customer_type                      25893 non-null  int64  
 2   age                                25893 non-null  float64
 3   class                              25893 non-null  float64
 4   flight_distance                    25893 non-null  float64
 5   inflight_wifi_service              25893 non-null  float64
 6   departure_arrival_time_convenient  25893 non-null  float64
 7   ease_of_online_booking             25893 non-null  float64
 8   gate_location                      25893 non-null  float64
 9   food_and_drink                     25893 non-null  float64
 10  online_boarding                    25893 non-null  float64
 11  seat_comfort                       25893 non-null  flo

In [48]:
y_teste.head()

,0
0,1
1,1
2,0
3,1
4,1


In [49]:
y_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25893 entries, 0 to 25892
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       25893 non-null  int64
dtypes: int64(1)
memory usage: 202.4 KB


In [50]:
y_teste_count = y_teste.value_counts()
y_teste_count

,count
0,
0,14528
1,11365


In [51]:
razao_classes(y_teste_count[0], y_teste_count[1])

Balanceado


#### 1.2.4 Comentários

Resumo quantos as classes:

| Classe      | Valor Absoluto | Porcentagem |
|-------------|----------------|-------------|
| Treino      | 72,515         | 56.0%       |
| Validação   | 31,079         | 24.0%       |
| Teste       | 25,893         | 20.0%       |
| **Total**   | **129,487**    | **100%**    |

Um total de 25 colunas (features) estão sendo consideradas e uma coluna target (0 ou 1). As features foram tratadas para serem convertidas em dados numéricos, sendo regularizadas e as features categóricas passaram por um processo de encoder.

Os dados foram tratados, não havendo nulos/N.A.'s ou erros de preenchimento.

Importante notar que não há desbalanceamento significativo entre as classes (valores 0 e 1 do target).

### 1.3 Análises

#### 1.3.1 Random Forest Classifier

In [52]:
rf_n_estimators = [50, 100, 150]
rf_max_depth = [None, 2, 4]
rf_treino_metricas = {}
rf_val_metricas = {}

for n in rf_n_estimators:
  for m in rf_max_depth:
    #treinamento do modelo variando parâmetros
    rf_model = RandomForestClassifier(n_estimators=n, max_depth=m)
    rf_model.fit(X_treino, y_treino_1d)

    #predição de treino
    Y_treino_pred = rf_model.predict(X_treino)

    #métricas de treino
    acuracia_treino = accuracy_score(y_treino_1d,Y_treino_pred)
    precisao_treino = precision_score(y_treino_1d,Y_treino_pred)
    recall_treino = recall_score(y_treino_1d,Y_treino_pred)
    f1_treino = f1_score(y_treino_1d,Y_treino_pred)

    #armazenando na biblioteca
    rf_treino_metricas.update({f"rf_n_estimators{n} e rf_max_depth{m}": {"acuracia_treino":acuracia_treino,
                                                                        "precisao_treino":precisao_treino,
                                                                        "recall_treino":recall_treino,
                                                                        "f1_treino":f1_treino}})
    #predição de validação
    Y_val_pred = rf_model.predict(X_val)

    #métricas de treino
    acuracia_val = accuracy_score(y_val_1d,Y_val_pred)
    precisao_val = precision_score(y_val_1d,Y_val_pred)
    recall_val = recall_score(y_val_1d,Y_val_pred)
    f1_val = f1_score(y_val_1d,Y_val_pred)

    #armazenando na biblioteca
    rf_val_metricas.update({f"rf_n_estimators{n} e rf_max_depth{m}": {"acuracia_val":acuracia_val,
                                                                        "precisao_val":precisao_val,
                                                                        "recall_val":recall_val,
                                                                        "f1_val":f1_val}})


In [53]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(rf_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(rf_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)

                                       acuracia_treino  precisao_treino  \
rf_n_estimators50 e rf_max_depthNone          0.999931         1.000000   
rf_n_estimators50 e rf_max_depth2             0.867558         0.870124   
rf_n_estimators50 e rf_max_depth4             0.908378         0.910712   
rf_n_estimators100 e rf_max_depthNone         1.000000         1.000000   
rf_n_estimators100 e rf_max_depth2            0.871599         0.883776   
rf_n_estimators100 e rf_max_depth4            0.911246         0.912654   
rf_n_estimators150 e rf_max_depthNone         1.000000         1.000000   
rf_n_estimators150 e rf_max_depth2            0.867476         0.873698   
rf_n_estimators150 e rf_max_depth4            0.908598         0.912124   

                                       recall_treino  f1_treino  acuracia_val  \
rf_n_estimators50 e rf_max_depthNone        0.999841   0.999920      0.964703   
rf_n_estimators50 e rf_max_depth2           0.816247   0.842325      0.868400   
rf_n_e

**Discussão dos resultados de treino e validação**

Os melhores parâmetros para colocar avançar para etapa de teste é aquele em que conseguimos equilibrar a performance (acuária, precisão, recall) e generalização (evitar overfitting). Portanto, a decisão será baseada nos parâmetros que tiver uma acurácia e f1_score (média harmônica entre Precisão e Recall) otimizados, assim contemplando todas as métricas, e que há uma diferença pequena entre seus valores no treino e validação indicando que não houve overfitting e que há boa capacidade de generalização pelo modelo.

**Melhores parâmetros:
n_estimators=50, max_depth=4**

Motivos:

Performance: F1_val=0.906 (equilíbrio entre precisão e recall).

Generalização: Diferença mínima entre treino (0.918) e validação (0.919).


In [54]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
rf_model_final = RandomForestClassifier(n_estimators=50, max_depth=4)
rf_model_final.fit(X_concat_treino_val, y_concat_treino_val)

#predição de teste
Y_teste_pred = rf_model_final.predict(X_teste)

#métricas de teste
acuracia_teste = accuracy_score(y_teste_1d,Y_teste_pred)
precisao_teste = precision_score(y_teste_1d,Y_teste_pred)
recall_teste = recall_score(y_teste_1d,Y_teste_pred)
f1_teste = f1_score(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"Random Forest Classifier":{
    "Acurácia": [acuracia_teste],
    "Precisão": [precisao_teste],
    "Recall": [recall_teste],
    "F1-Score": [f1_teste]}
}

# Converter para DataFrame
rf_teste_resultados = pd.DataFrame(metricas_teste)
rf_teste_resultados.head()

,Random Forest Classifier
Acurácia,[0.9229135287529449]
Precisão,[0.911896597203904]
Recall,[0.9125384953805543]
F1-Score,[0.9122174333714487]


#### 1.3.2 K-Neighbors Classifier

In [55]:
kn_n_neighbors = [2, 3, 4, 5, 6, 7, 8]
kn_treino_metricas = {}
kn_val_metricas = {}

for n in kn_n_neighbors:
    #treinamento do modelo variando parâmetros
    kn_model = KNeighborsClassifier(n_neighbors=n)
    kn_model.fit(X_treino, y_treino_1d)

    #predição de treino
    Y_treino_pred = kn_model.predict(X_treino)

    #métricas de treino
    acuracia_treino = accuracy_score(y_treino_1d,Y_treino_pred)
    precisao_treino = precision_score(y_treino_1d,Y_treino_pred)
    recall_treino = recall_score(y_treino_1d,Y_treino_pred)
    f1_treino = f1_score(y_treino_1d,Y_treino_pred)

    #armazenando na biblioteca
    kn_treino_metricas.update({f"kn_n_neighbors{n}": {"acuracia_treino":acuracia_treino,
                                                      "precisao_treino":precisao_treino,
                                                      "recall_treino":recall_treino,
                                                      "f1_treino":f1_treino}})

    #predição de validação
    Y_val_pred = kn_model.predict(X_val)

    #métricas de treino
    acuracia_val = accuracy_score(y_val_1d,Y_val_pred)
    precisao_val = precision_score(y_val_1d,Y_val_pred)
    recall_val = recall_score(y_val_1d,Y_val_pred)
    f1_val = f1_score(y_val_1d,Y_val_pred)

    #armazenando na biblioteca
    kn_val_metricas.update({f"kn_n_neighbors{n}": {"acuracia_val":acuracia_val,
                                                   "precisao_val":precisao_val,
                                                   "recall_val":recall_val,
                                                   "f1_val":f1_val}})

In [56]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(kn_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(kn_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)

                 acuracia_treino  precisao_treino  recall_treino  f1_treino  \
kn_n_neighbors2         0.840199         1.000000       0.631284   0.773972   
kn_n_neighbors3         0.832186         0.812008       0.797410   0.804643   
kn_n_neighbors4         0.784665         0.852670       0.608247   0.710012   
kn_n_neighbors5         0.781562         0.755893       0.732563   0.744046   
kn_n_neighbors6         0.757471         0.792164       0.597047   0.680904   
kn_n_neighbors7         0.756312         0.731249       0.692090   0.711131   
kn_n_neighbors8         0.742177         0.762429       0.588488   0.664260   

                 acuracia_val  precisao_val  recall_val    f1_val  
kn_n_neighbors2      0.661283      0.683921    0.406118  0.509619  
kn_n_neighbors3      0.676277      0.627851    0.621278  0.624548  
kn_n_neighbors4      0.668715      0.669661    0.464919  0.548817  
kn_n_neighbors5      0.675665      0.631775    0.603163  0.617138  
kn_n_neighbors6      0.6705

**Discussão dos resultados de treino e validação**

Os melhores parâmetros para colocar avançar para etapa de teste é aquele em que conseguimos equilibrar a performance (acuária, precisão, recall) e generalização (evitar overfitting). Portanto, a decisão será baseada nos parâmetros que tiver uma acurácia e f1_score (média harmônica entre Precisão e Recall) otimizados, assim contemplando todas as métricas, e que há uma diferença pequena entre seus valores no treino e validação indicando que não houve overfitting e que há boa capacidade de generalização pelo modelo.

**Melhores parâmetros:
n_neighbors=3**

Motivos:
Melhor F1-Score em validação (0.6245): Equilíbrio entre precisão e recall.

Generalização: Diferença moderada entre treino (0.805) e validação (0.625).

Simplicidade: Número pequeno de vizinhos (mais eficiente computacionalmente).

In [57]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
kn_model_final = KNeighborsClassifier(n_neighbors=3)
kn_model_final.fit(X_concat_treino_val, y_concat_treino_val)

#predição de teste
Y_teste_pred = kn_model_final.predict(X_teste)

#métricas de teste
acuracia_teste = accuracy_score(y_teste_1d,Y_teste_pred)
precisao_teste = precision_score(y_teste_1d,Y_teste_pred)
recall_teste = recall_score(y_teste_1d,Y_teste_pred)
f1_teste = f1_score(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"K-Neighbors Classifier":{
    "Acurácia": [acuracia_teste],
    "Precisão": [precisao_teste],
    "Recall": [recall_teste],
    "F1-Score": [f1_teste]}
}

# Converter para DataFrame
kn_teste_resultados = pd.DataFrame(metricas_teste)
kn_teste_resultados.head()

,K-Neighbors Classifier
Acurácia,[0.6884486154559147]
Precisão,[0.6480251346499102]
Recall,[0.6351957765068191]
F1-Score,[0.6415463230393246]


#### 1.3.3 Logistic Regression

In [58]:
lr_C = [1.0, 2.0, 3.0, 4.0, 5.0]
lr_solver = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
lr_max_iter = [100, 500, 1000, 2000]
lr_treino_metricas = {}
lr_val_metricas = {}

for c in lr_C:
  for s in lr_solver:
    for i in lr_max_iter:
      #treinamento do modelo variando parâmetros
      lr_model = LogisticRegression(C=c, solver=s, max_iter=i)
      lr_model.fit(X_treino, y_treino_1d)

      #predição de treino
      Y_treino_pred = lr_model.predict(X_treino)

      #métricas de treino
      acuracia_treino = accuracy_score(y_treino_1d,Y_treino_pred)
      precisao_treino = precision_score(y_treino_1d,Y_treino_pred)
      recall_treino = recall_score(y_treino_1d,Y_treino_pred)
      f1_treino = f1_score(y_treino_1d,Y_treino_pred)

      #armazenando na biblioteca
      lr_treino_metricas.update({f"lr_C{c}, lr_solver{s} e lr_max_iter{i}":
                                                        {"acuracia_treino":acuracia_treino,
                                                        "precisao_treino":precisao_treino,
                                                        "recall_treino":recall_treino,
                                                        "f1_treino":f1_treino}})

      #predição de validação
      Y_val_pred = lr_model.predict(X_val)

      #métricas de treino
      acuracia_val = accuracy_score(y_val_1d,Y_val_pred)
      precisao_val = precision_score(y_val_1d,Y_val_pred)
      recall_val = recall_score(y_val_1d,Y_val_pred)
      f1_val = f1_score(y_val_1d,Y_val_pred)

      #armazenando na biblioteca
      lr_treino_metricas.update({f"lr_C{c}, lr_solver{s} e lr_max_iter{i}":
                                                        {"acuracia_treino":acuracia_treino,
                                                        "precisao_treino":precisao_treino,
                                                        "recall_treino":recall_treino,
                                                        "f1_treino":f1_treino}})

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [59]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(lr_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(lr_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)

                                              acuracia_treino  \
lr_C1.0, lr_solverlbfgs e lr_max_iter100             0.872606   
lr_C1.0, lr_solverlbfgs e lr_max_iter500             0.874054   
lr_C1.0, lr_solverlbfgs e lr_max_iter1000            0.875543   
lr_C1.0, lr_solverlbfgs e lr_max_iter2000            0.876439   
lr_C1.0, lr_solverliblinear e lr_max_iter100         0.793132   
...                                                       ...   
lr_C5.0, lr_solversag e lr_max_iter2000              0.573150   
lr_C5.0, lr_solversaga e lr_max_iter100              0.566724   
lr_C5.0, lr_solversaga e lr_max_iter500              0.568034   
lr_C5.0, lr_solversaga e lr_max_iter1000             0.567731   
lr_C5.0, lr_solversaga e lr_max_iter2000             0.569910   

                                              precisao_treino  recall_treino  \
lr_C1.0, lr_solverlbfgs e lr_max_iter100             0.860954       0.842052   
lr_C1.0, lr_solverlbfgs e lr_max_iter500             0.8667

**Discussão dos resultados de treino e validação**

Os melhores parâmetros para colocar avançar para etapa de teste é aquele em que conseguimos equilibrar a performance (acuária, precisão, recall) e generalização (evitar overfitting). Portanto, a decisão será baseada nos parâmetros que tiver uma acurácia e f1_score (média harmônica entre Precisão e Recall) otimizados, assim contemplando todas as métricas, e que há uma diferença pequena entre seus valores no treino e validação indicando que não houve overfitting e que há boa capacidade de generalização pelo modelo.

**Melhores parâmetros:
lr_C1.0, lr_solverlbfgs e lr_max_iter2000**

Motivos:

Alto F1-Score (0.8546), indicando bom balanço entre precisão e recall.

Acurácia (0.8764) e Precisão (0.8720) consistentemente altas.

Convergência garantida:

Com max_iter=2000, o otimizador LBFGS teve iterações suficientes para convergir (evitando os warnings de não-convergência).

Regularização adequada:

C=1.0 (valor padrão) evita overfitting sem subajustar os dados.

Solver eficiente:

lbfgs é recomendado para datasets médios/pequenos e lida bem com features escalonadas.



In [60]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
lr_model_final = LogisticRegression(C=1.0, solver='lbfgs', max_iter=2000)
lr_model_final.fit(X_concat_treino_val, y_concat_treino_val)

#predição de teste
Y_teste_pred = lr_model_final.predict(X_teste)

#métricas de teste
acuracia_teste = accuracy_score(y_teste_1d,Y_teste_pred)
precisao_teste = precision_score(y_teste_1d,Y_teste_pred)
recall_teste = recall_score(y_teste_1d,Y_teste_pred)
f1_teste = f1_score(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"Logistic Regression":{
    "Acurácia": [acuracia_teste],
    "Precisão": [precisao_teste],
    "Recall": [recall_teste],
    "F1-Score": [f1_teste]}
}

# Converter para DataFrame
lr_teste_resultados = pd.DataFrame(metricas_teste)
lr_teste_resultados.head()

,Logistic Regression
Acurácia,[0.8712779515699224]
Precisão,[0.8674290942360475]
Recall,[0.8342278926528817]
F1-Score,[0.8505045974433729]


#### 1.3.4 Decision Tree Classifier

In [61]:
dt_max_depth = [None, 3, 5, 7, 10]
dt_treino_metricas = {}
dt_val_metricas = {}

for n in dt_max_depth:
    #treinamento do modelo variando parâmetros
    dt_model = DecisionTreeClassifier(max_depth=n)
    dt_model.fit(X_treino, y_treino_1d)

    #predição de treino
    Y_treino_pred = dt_model.predict(X_treino)

    #métricas de treino
    acuracia_treino = accuracy_score(y_treino_1d,Y_treino_pred)
    precisao_treino = precision_score(y_treino_1d,Y_treino_pred)
    recall_treino = recall_score(y_treino_1d,Y_treino_pred)
    f1_treino = f1_score(y_treino_1d,Y_treino_pred)

    #armazenando na biblioteca
    dt_treino_metricas.update({f"dt_max_depth{n}": {"acuracia_treino":acuracia_treino,
                                                      "precisao_treino":precisao_treino,
                                                      "recall_treino":recall_treino,
                                                      "f1_treino":f1_treino}})

    #predição de validação
    Y_val_pred = dt_model.predict(X_val)

    #métricas de treino
    acuracia_val = accuracy_score(y_val_1d,Y_val_pred)
    precisao_val = precision_score(y_val_1d,Y_val_pred)
    recall_val = recall_score(y_val_1d,Y_val_pred)
    f1_val = f1_score(y_val_1d,Y_val_pred)

    #armazenando na biblioteca
    dt_val_metricas.update({f"dt_max_depth{n}": {"acuracia_val":acuracia_val,
                                                   "precisao_val":precisao_val,
                                                   "recall_val":recall_val,
                                                   "f1_val":f1_val}})

In [62]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(dt_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(dt_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)

                  acuracia_treino  precisao_treino  recall_treino  f1_treino  \
dt_max_depthNone         1.000000         1.000000       1.000000   1.000000   
dt_max_depth3            0.884162         0.839506       0.905912   0.871446   
dt_max_depth5            0.906764         0.907732       0.873680   0.890381   
dt_max_depth7            0.934510         0.952662       0.893280   0.922016   
dt_max_depth10           0.955457         0.963691       0.932353   0.947763   

                  acuracia_val  precisao_val  recall_val    f1_val  
dt_max_depthNone      0.946105      0.936491    0.939342  0.937915  
dt_max_depth3         0.885453      0.840398    0.908159  0.872966  
dt_max_depth5         0.906689      0.906531    0.874898  0.890434  
dt_max_depth7         0.933717      0.952630    0.891380  0.920988  
dt_max_depth10        0.947392      0.954455    0.922637  0.938276  


**Discussão dos resultados de treino e validação**

Os melhores parâmetros para colocar avançar para etapa de teste é aquele em que conseguimos equilibrar a performance (acuária, precisão, recall) e generalização (evitar overfitting). Portanto, a decisão será baseada nos parâmetros que tiver uma acurácia e f1_score (média harmônica entre Precisão e Recall) otimizados, assim contemplando todas as métricas, e que há uma diferença pequena entre seus valores no treino e validação indicando que não houve overfitting e que há boa capacidade de generalização pelo modelo.

**Melhores parâmetros:
dt_max_depth 10**

Motivos:
apresenta o melhor equilíbrio;

Maior F1-score (0.9381) → Melhor balanço entre precisão e recall.

Acurácia e Precisão altas → Boa capacidade de classificação correta.

Recall elevado (0.9223) → Detecção eficiente de casos positivos.

In [63]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
dt_model_final = DecisionTreeClassifier(max_depth=10)
dt_model_final.fit(X_concat_treino_val, y_concat_treino_val)

#predição de teste
Y_teste_pred = dt_model_final.predict(X_teste)

#métricas de teste
acuracia_teste = accuracy_score(y_teste_1d,Y_teste_pred)
precisao_teste = precision_score(y_teste_1d,Y_teste_pred)
recall_teste = recall_score(y_teste_1d,Y_teste_pred)
f1_teste = f1_score(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"Decision Tree Classifier":{
    "Acurácia": [acuracia_teste],
    "Precisão": [precisao_teste],
    "Recall": [recall_teste],
    "F1-Score": [f1_teste]}
}

# Converter para DataFrame
dt_teste_resultados = pd.DataFrame(metricas_teste)
dt_teste_resultados.head()

,Decision Tree Classifier
Acurácia,[0.9480168385277874]
Precisão,[0.9481971906593898]
Recall,[0.9325120985481742]
F1-Score,[0.9402892378670925]


### 1.4 Resultados e Discussão

In [69]:
# Função para reformatar cada DataFrame
def formatar_dataframe(df, nome_modelo):
    # Extrai os valores das listas e cria um novo DataFrame
    df_reformatado = pd.DataFrame({
        'Métrica': df.index,  # Pega os nomes das métricas (Acurácia, Precisão, etc.)
        'Valor': df.iloc[:, 0].str[0],  # Extrai o valor dentro da lista
        'Modelo': nome_modelo
    })
    return df_reformatado

# Lista de DataFrames originais e seus nomes
dataframes_originais = [rf_teste_resultados, kn_teste_resultados, lr_teste_resultados, dt_teste_resultados]
nomes_modelos = ['Random Forest', 'K-Neighbors', 'Logistic Regression', 'Decision Tree']

# Reformata e concatena todos
resultados_finais = pd.concat(
    [formatar_dataframe(df, nome) for df, nome in zip(dataframes_originais, nomes_modelos)],
    ignore_index=True
)

# Opcional: Converter para formato wide (métricas como colunas)
resultados_pivot = resultados_finais.pivot(
    index='Modelo',
    columns='Métrica',
    values='Valor'
).reset_index()

print(resultados_pivot)

Métrica               Modelo  Acurácia  F1-Score  Precisão    Recall
0              Decision Tree  0.948017  0.940289  0.948197  0.932512
1                K-Neighbors  0.688449  0.641546  0.648025  0.635196
2        Logistic Regression  0.871278  0.850505  0.867429  0.834228
3              Random Forest  0.922914  0.912217  0.911897  0.912538


**Melhor Modelo: Decision Tree**

Acurácia mais alta (0.9480): Classifica corretamente 94.8% das amostras.

F1-Score mais alto (0.9403): Equilíbrio ideal entre Precisão e Recall, indicando robustez contra desbalanceamento de classes.

*Precisão e Recall consistentes:*

Precisão (0.9482): Baixa taxa de falsos positivos.

Recall (0.9325): Captura a maioria dos casos positivos.

## 2.0 Ensaio de Regressão

### 2.1 Importando e lendo o dataset

In [2]:
# Treino
X_treino = pd.read_csv('Ensaio_de_Regressao/X_training.csv')
y_treino = pd.read_csv('Ensaio_de_Regressao/y_training.csv')

# Validação
X_val = pd.read_csv('Ensaio_de_Regressao/X_validation.csv')
y_val = pd.read_csv('Ensaio_de_Regressao/y_val.csv')

# Teste
X_teste = pd.read_csv('Ensaio_de_Regressao/X_test.csv')
y_teste = pd.read_csv('Ensaio_de_Regressao/y_test.csv')

# Corrigindo os rótulos
y_treino_1d = y_treino.to_numpy().ravel()
y_val_1d = y_val.to_numpy().ravel()
y_teste_1d = y_teste.to_numpy().ravel()

# Treino + Validação
X_concat_treino_val = pd.concat([X_treino, X_val], axis=0)
y_concat_treino_val = np.concatenate([y_treino_1d, y_val_1d])

### 2.2 Explorando o dataset

#### 2.2.1 Treino

In [73]:
X_treino.head()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,0.205673,0.092100,0.720,0.802,0.000000,0.090909,0.694,0.431778,1,0.0582,0.103876,0.8,0.723
1,-0.240409,0.737000,0.483,0.412,0.000000,0.636364,0.116,-0.262732,1,0.0402,1.711532,0.6,0.247
2,-0.125770,0.274000,0.671,0.565,0.000065,1.000000,0.370,0.013612,0,0.1600,1.009176,0.8,0.561
3,-0.580967,0.002340,0.704,0.529,0.874000,1.000000,0.370,-0.266382,0,0.0416,0.666173,0.8,0.507
4,-0.688566,0.000414,0.354,0.910,0.205000,0.090909,0.456,0.422914,1,0.0430,1.182630,0.8,0.362


In [75]:
X_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10547 entries, 0 to 10546
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_duration_ms  10547 non-null  float64
 1   acousticness      10547 non-null  float64
 2   danceability      10547 non-null  float64
 3   energy            10547 non-null  float64
 4   instrumentalness  10547 non-null  float64
 5   key               10547 non-null  float64
 6   liveness          10547 non-null  float64
 7   loudness          10547 non-null  float64
 8   audio_mode        10547 non-null  int64  
 9   speechiness       10547 non-null  float64
 10  tempo             10547 non-null  float64
 11  time_signature    10547 non-null  float64
 12  audio_valence     10547 non-null  float64
dtypes: float64(12), int64(1)
memory usage: 1.0 MB


In [89]:
X_treino.describe()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
count,1.054700e+04,10547.000000,10547.000000,10547.000000,10547.000000,10547.000000,10547.000000,1.054700e+04,10547.000000,10547.000000,1.054700e+04,10547.000000,10547.000000
mean,5.995857e-17,0.259502,0.633974,0.645438,0.079089,0.483196,0.180416,2.489291e-16,0.627098,0.100937,-9.499054e-17,0.791581,0.529879
std,1.000047e+00,0.290173,0.156223,0.214348,0.222818,0.327242,0.144705,1.000047e+00,0.483599,0.103352,1.000047e+00,0.060822,0.244461
min,-3.553790e+00,0.000001,0.000000,0.001070,0.000000,0.000000,0.015700,-7.632256e+00,0.000000,0.000000,-4.220028e+00,0.000000,0.000000
25%,-5.810364e-01,0.024600,0.533000,0.512000,0.000000,0.181818,0.092700,-4.243671e-01,0.000000,0.037850,-7.756112e-01,0.800000,0.336000
50%,-1.146491e-01,0.132000,0.645000,0.675000,0.000011,0.454545,0.122000,2.346876e-01,1.000000,0.055200,-3.656695e-02,0.800000,0.532000
75%,4.224443e-01,0.432000,0.748000,0.815000,0.002835,0.727273,0.224000,6.609353e-01,1.000000,0.116000,6.566737e-01,0.800000,0.727000
max,1.969099e+01,0.996000,0.987000,0.999000,0.989000,1.000000,0.984000,2.292930e+00,1.000000,0.941000,4.226719e+00,1.000000,0.982000


In [74]:
y_treino.head()

,song_popularity
0,79.0
1,86.0
2,63.0
3,25.0
4,35.0


In [76]:
y_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10547 entries, 0 to 10546
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   song_popularity  10547 non-null  float64
dtypes: float64(1)
memory usage: 82.5 KB


In [90]:
y_treino.describe()

,song_popularity
count,10547.000000
mean,53.046961
std,21.864535
min,0.100000
25%,40.000000
50%,56.000000
75%,69.000000
max,100.000000


#### 2.2.2 Validação

In [77]:
X_val.head()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,0.143252,0.025900,0.616,0.933,0.00000,0.000000,0.3590,0.962307,0,0.0513,0.309853,0.8,0.806
1,0.363603,0.000188,0.490,0.972,0.02990,0.909091,0.3680,0.765216,0,0.1110,-0.908089,0.8,0.376
2,-1.409083,0.694000,0.876,0.167,0.91200,1.000000,0.3690,-1.733352,1,0.0885,-0.975052,0.8,0.845
3,0.270931,0.003750,0.770,0.801,0.00281,0.818182,0.0515,0.852812,0,0.0336,0.338018,0.8,0.725
4,0.367789,0.011600,0.596,0.869,0.17300,0.909091,0.0678,-1.177535,1,0.0370,-0.389052,0.8,0.944


In [78]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_duration_ms  4521 non-null   float64
 1   acousticness      4521 non-null   float64
 2   danceability      4521 non-null   float64
 3   energy            4521 non-null   float64
 4   instrumentalness  4521 non-null   float64
 5   key               4521 non-null   float64
 6   liveness          4521 non-null   float64
 7   loudness          4521 non-null   float64
 8   audio_mode        4521 non-null   int64  
 9   speechiness       4521 non-null   float64
 10  tempo             4521 non-null   float64
 11  time_signature    4521 non-null   float64
 12  audio_valence     4521 non-null   float64
dtypes: float64(12), int64(1)
memory usage: 459.3 KB


In [87]:
X_val.describe()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,0.031180,0.256280,0.633485,0.647265,0.078201,0.477851,0.179284,0.014352,0.635036,0.103951,0.024200,0.792214,0.525134
std,1.056152,0.285704,0.156141,0.213404,0.223037,0.330599,0.143020,0.985424,0.481473,0.106096,1.004350,0.057772,0.246369
min,-3.308429,0.000001,0.068400,0.001070,0.000000,0.000000,0.010900,-7.282915,0.000000,0.022400,-2.595954,0.200000,0.027700
25%,-0.564346,0.025200,0.534000,0.511000,0.000000,0.181818,0.093000,-0.420717,0.000000,0.037800,-0.768134,0.800000,0.332000
50%,-0.088134,0.133000,0.646000,0.675000,0.000010,0.454545,0.123000,0.243551,1.000000,0.056600,-0.036079,0.800000,0.517000
75%,0.476425,0.411000,0.748000,0.819000,0.002270,0.727273,0.220000,0.669799,1.000000,0.124000,0.660282,0.800000,0.726000
max,17.576173,0.996000,0.978000,0.997000,0.997000,1.000000,0.975000,2.356280,1.000000,0.915000,3.174667,1.000000,0.982000


In [79]:
y_val.head()

,song_popularity
0,60.0
1,48.0
2,43.0
3,59.0
4,79.0


In [80]:
y_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   song_popularity  4521 non-null   float64
dtypes: float64(1)
memory usage: 35.4 KB


In [88]:
y_val.describe()

,song_popularity
count,4521.000000
mean,53.028423
std,21.854456
min,0.100000
25%,40.000000
50%,56.000000
75%,69.000000
max,100.000000


#### 2.2.3 Teste

In [81]:
X_teste.head()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,-1.662539,0.073,0.754,0.342,0.000000,0.909091,0.1930,-0.319043,0,0.0991,0.276006,0.8,0.423
1,-0.054995,0.191,0.687,0.792,0.000000,0.454545,0.1670,1.226398,1,0.0452,-0.733694,0.8,0.671
2,0.564739,0.318,0.630,0.478,0.000065,1.000000,0.0942,-0.817506,1,0.0275,-0.995967,0.8,0.279
3,0.094512,0.011,0.363,0.824,0.000000,0.363636,0.1050,1.078580,1,0.0405,-0.083765,0.8,0.164
4,-0.018639,0.334,0.334,0.617,0.000000,0.545455,0.1630,0.422132,0,0.3830,-1.249176,0.8,0.603


In [82]:
X_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3767 entries, 0 to 3766
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_duration_ms  3767 non-null   float64
 1   acousticness      3767 non-null   float64
 2   danceability      3767 non-null   float64
 3   energy            3767 non-null   float64
 4   instrumentalness  3767 non-null   float64
 5   key               3767 non-null   float64
 6   liveness          3767 non-null   float64
 7   loudness          3767 non-null   float64
 8   audio_mode        3767 non-null   int64  
 9   speechiness       3767 non-null   float64
 10  tempo             3767 non-null   float64
 11  time_signature    3767 non-null   float64
 12  audio_valence     3767 non-null   float64
dtypes: float64(12), int64(1)
memory usage: 382.7 KB


In [86]:
X_teste.describe()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
count,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000
mean,0.026829,0.258553,0.631432,0.641028,0.074751,0.477810,0.177946,-0.009692,0.622777,0.103130,-0.027278,0.792036,0.526013
std,1.106926,0.288297,0.158828,0.214247,0.216369,0.330017,0.143134,1.006936,0.484756,0.105142,0.998838,0.058846,0.243029
min,-3.140069,0.000002,0.059400,0.002050,0.000000,0.000000,0.014800,-8.163827,0.000000,0.023100,-2.330265,0.200000,0.023000
25%,-0.546488,0.021550,0.530000,0.502500,0.000000,0.181818,0.093100,-0.390215,0.000000,0.037400,-0.809650,0.800000,0.338500
50%,-0.104272,0.132000,0.640000,0.672000,0.000013,0.454545,0.121000,0.213049,1.000000,0.055500,-0.037404,0.800000,0.522000
75%,0.442776,0.420000,0.748000,0.810000,0.002300,0.727273,0.217000,0.657937,1.000000,0.121000,0.635009,0.800000,0.719000
max,27.360187,0.996000,0.975000,0.996000,0.975000,1.000000,0.986000,1.750409,1.000000,0.906000,3.064236,1.000000,0.984000


In [84]:
y_teste.head()

,song_popularity
0,87.0
1,100.0
2,37.0
3,0.1
4,78.0


In [83]:
y_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3767 entries, 0 to 3766
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   song_popularity  3767 non-null   float64
dtypes: float64(1)
memory usage: 29.6 KB


In [85]:
y_teste.describe()

,song_popularity
count,3767.000000
mean,52.801009
std,22.068762
min,0.100000
25%,40.000000
50%,55.000000
75%,69.000000
max,100.000000


#### 2.2.4 Comentários

Resumo quanto as classes:

| Classe     | Valor Absoluto | Porcentagem |
|------------|----------------|-------------|
| Treino     | 10,547         | 56.0%       |
| Validação  | 4,521          | 24.0%       |
| Teste      | 3,766          | 20.0%       |
| **Total**  | **18,834**     | **100%**    |

Há um total de 13 colunas (Features), todas em valores numéricos, referentes a características métricas de uma música.

Análise de Balanceamento da Variável y (Target)

Para avaliar se há balanceamento na variável alvo (y), comparamos as estatísticas (média e desvio padrão) entre os conjuntos:

| Conjunto   | Média (y) | Desvio Padrão (y) |
|------------|-----------|-------------------|
| Treino     | 53.047    | 21.865            |
| Validação  | 53.028    | 21.854            |
| Teste      | 52.801    | 22.069            |

Diferença de apenas ~0.246 entre treino e teste (53.047 vs. 52.801).

Validação quase idêntica ao treino (53.028 vs. 53.047).

Todos em torno de ~21.8 a 22.1, indicando dispersão similar.

Há balanceamento entre os conjuntos, pois as distribuições de y são estatisticamente similares (médias e variâncias próximas).

### 2.3 Análises

#### 2.3.1 Decision Tree Regressor

In [3]:
dtr_max_depth = [None, 2, 4, 6, 8, 10]
dtr_treino_metricas = {}
dtr_val_metricas = {}

for n in dtr_max_depth:
    #treinamento do modelo variando parâmetros
    dtr_model = DecisionTreeRegressor(max_depth=n)
    dtr_model.fit(X_treino, y_treino_1d)

    #predição de treino
    Y_treino_pred = dtr_model.predict(X_treino)

    #métricas de treino
    r2_treino = r2_score(y_treino_1d,Y_treino_pred)
    MSE_treino = mean_squared_error(y_treino_1d,Y_treino_pred)
    RMSE_treino = root_mean_squared_error(y_treino_1d,Y_treino_pred)
    MAE_treino = mean_absolute_error(y_treino_1d,Y_treino_pred)
    MAPE_treino = mean_absolute_percentage_error(y_treino_1d, Y_treino_pred)

    #armazenando na biblioteca
    dtr_treino_metricas.update({f"dtr_max_depth{n}": {"r2_treino":r2_treino,
                                                      "MSE_treino":MSE_treino,
                                                      "RMSE_treino":RMSE_treino,
                                                      "MAE_treino":MAE_treino,
                                                      "MAPE_treino":MAPE_treino}})
    #predição de validação
    Y_val_pred = dtr_model.predict(X_val)

    #métricas de treino
    r2_val = r2_score(y_val_1d,Y_val_pred)
    MSE_val = mean_squared_error(y_val_1d,Y_val_pred)
    RMSE_val = root_mean_squared_error(y_val_1d,Y_val_pred)
    MAE_val = mean_absolute_error(y_val_1d,Y_val_pred)
    MAPE_val = mean_absolute_percentage_error(y_val_1d, Y_val_pred)

    #armazenando na biblioteca
    dtr_val_metricas.update({f"dtr_max_depth{n}": {"r2_val":r2_val,
                                                    "MSE_val":MSE_val,
                                                    "RMSE_val":RMSE_val,
                                                    "MAE_val":MAE_val,
                                                    "MAPE_val":MAPE_val}})

In [4]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(dtr_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(dtr_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)

                   r2_treino  MSE_treino  RMSE_treino  MAE_treino  \
dtr_max_depthNone   0.991757    3.940403     1.985045    0.214099   
dtr_max_depth2      0.043334  457.298367    21.384536   16.986509   
dtr_max_depth4      0.086649  436.593329    20.894816   16.618123   
dtr_max_depth6      0.144675  408.855894    20.220185   16.026218   
dtr_max_depth8      0.246287  360.284222    18.981154   14.820318   
dtr_max_depth10     0.384623  294.157782    17.151029   12.931231   

                   MAPE_treino    r2_val     MSE_val   RMSE_val    MAE_val  \
dtr_max_depthNone     8.262787 -0.316818  628.795856  25.075802  17.249829   
dtr_max_depth2      850.093201  0.037609  459.552983  21.437187  16.985102   
dtr_max_depth4      826.874283  0.062168  447.825853  21.161896  16.849288   
dtr_max_depth6      730.789577  0.063592  447.145794  21.145822  16.743250   
dtr_max_depth8      616.858839  0.037286  459.707225  21.440784  16.865962   
dtr_max_depth10     486.638711 -0.007999  481.33

**Discussão dos Resultados de Treino e Validação para Métricas de Regressão**

Na seleção do melhor modelo de regressão, o equilíbrio entre performance (capacidade de prever corretamente) e generalização (adaptação a novos dados) é fundamental. As métricas utilizadas (R², MSE, RMSE, MAE e MAPE) fornecem insights complementares sobre o comportamento do modelo, permitindo identificar se ele está sofrendo de overfitting ou underfitting.

**Melhores parâmetros: max_depth=6**

Motivos:
Maior R² na validação (0.064) entre todas as opções.

Menor MAE na validação (16.74).

Diferença controlada de MSE (38.29), indicando equilíbrio entre performance e generalização.

MAPE mais baixo (832.6%) que max_depth=2 e 4.

In [5]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
dtr_model_final = DecisionTreeRegressor(max_depth=6)
dtr_model_final.fit(X_concat_treino_val, y_concat_treino_val)

#predição de teste
Y_teste_pred = dtr_model_final.predict(X_teste)

#métricas de teste
r2_teste = r2_score(y_teste_1d,Y_teste_pred)
MSE_teste = mean_squared_error(y_teste_1d,Y_teste_pred)
RMSE_teste = root_mean_squared_error(y_teste_1d,Y_teste_pred)
MAE_teste = mean_absolute_error(y_teste_1d,Y_teste_pred)
MAPE_teste = mean_absolute_percentage_error(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"Decision Tree Regressor":{"r2_teste":[r2_teste],
                                             "MSE_treino":[MSE_teste],
                                             "RMSE_treino":[RMSE_teste],
                                             "MAE_treino":[MAE_teste],
                                             "MAPE_treino":[MAPE_teste]}
}

# Converter para DataFrame
dtr_teste_resultados = pd.DataFrame(metricas_teste)
dtr_teste_resultados.head()

,Decision Tree Regressor
r2_teste,[0.10171378835902489]
MSE_treino,[437.376433073918]
RMSE_treino,[20.913546640250143]
MAE_treino,[16.67987641901041]
MAPE_treino,[772.8304533737748]


#### 2.3.2 Polinomial Regression

In [6]:
pr_degree = [2, 3, 4]
pr_treino_metricas = {}
pr_val_metricas = {}

for n in pr_degree:
    #treinamento do modelo variando parâmetros
    pr = PolynomialFeatures(degree=n, include_bias=False)
    pr_features_treino = pr.fit_transform(X_treino)
    pr_model = LinearRegression()
    pr_model.fit(pr_features_treino, y_treino_1d)

    #predição de treino
    Y_treino_pred = pr_model.predict(pr_features_treino)

    #métricas de treino
    r2_treino = r2_score(y_treino_1d,Y_treino_pred)
    MSE_treino = mean_squared_error(y_treino_1d,Y_treino_pred)
    RMSE_treino = root_mean_squared_error(y_treino_1d,Y_treino_pred)
    MAE_treino = mean_absolute_error(y_treino_1d,Y_treino_pred)
    MAPE_treino = mean_absolute_percentage_error(y_treino_1d, Y_treino_pred)

    #armazenando na biblioteca
    pr_treino_metricas.update({f"pr_degree{n}": {"r2_treino":r2_treino,
                                                      "MSE_treino":MSE_treino,
                                                      "RMSE_treino":RMSE_treino,
                                                      "MAE_treino":MAE_treino,
                                                      "MAPE_treino":MAPE_treino}})
    #predição de validação
    pr_features_val = pr.transform(X_val)
    Y_val_pred = pr_model.predict(pr_features_val)

    #métricas de treino
    r2_val = r2_score(y_val_1d,Y_val_pred)
    MSE_val = mean_squared_error(y_val_1d,Y_val_pred)
    RMSE_val = root_mean_squared_error(y_val_1d,Y_val_pred)
    MAE_val = mean_absolute_error(y_val_1d,Y_val_pred)
    MAPE_val = mean_absolute_percentage_error(y_val_1d, Y_val_pred)

    #armazenando na biblioteca
    pr_val_metricas.update({f"pr_degree{n}": {"r2_val":r2_val,
                                                    "MSE_val":MSE_val,
                                                    "RMSE_val":RMSE_val,
                                                    "MAE_val":MAE_val,
                                                    "MAPE_val":MAPE_val}})

In [7]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(pr_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(pr_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)

            r2_treino  MSE_treino  RMSE_treino  MAE_treino  MAPE_treino  \
pr_degree2   0.094195  432.986210    20.808321   16.458032   835.053982   
pr_degree3   0.154418  404.198950    20.104700   15.883592   780.018145   
pr_degree4   0.333957  318.377086    17.843124   13.614247   591.339145   

                r2_val       MSE_val    RMSE_val    MAE_val     MAPE_val  
pr_degree2    0.066477    445.768223   21.113224  16.749939   854.793103  
pr_degree3   -0.047778    500.326254   22.367974  17.087201   867.828257  
pr_degree4 -102.923632  49624.740861  222.766112  36.104220  1018.480150  


**Discussão dos Resultados de Treino e Validação para Métricas de Regressão**

Na seleção do melhor modelo de regressão, o equilíbrio entre performance (capacidade de prever corretamente) e generalização (adaptação a novos dados) é fundamental. As métricas utilizadas (R², MSE, RMSE, MAE e MAPE) fornecem insights complementares sobre o comportamento do modelo, permitindo identificar se ele está sofrendo de overfitting ou underfitting.

**Melhores parâmetros: pr_degree=2**

Motivos:

É o único que não sofre overfitting severo.

Mantém R² positivo na validação (ainda que baixo).

Tem os menores erros (MSE, MAE) na validação.

In [9]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
pr = PolynomialFeatures(degree=2, include_bias=False)
pr_features_concat = pr.fit_transform(X_concat_treino_val)
pr_model_final = LinearRegression()
pr_model_final.fit(pr_features_concat, y_concat_treino_val)

#predição de teste
pr_features_teste = pr.transform(X_teste)
Y_teste_pred = pr_model_final.predict(pr_features_teste)

#métricas de teste
r2_teste = r2_score(y_teste_1d,Y_teste_pred)
MSE_teste = mean_squared_error(y_teste_1d,Y_teste_pred)
RMSE_teste = root_mean_squared_error(y_teste_1d,Y_teste_pred)
MAE_teste = mean_absolute_error(y_teste_1d,Y_teste_pred)
MAPE_teste = mean_absolute_percentage_error(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"Polinomial Regression":{"r2_teste":[r2_teste],
                                             "MSE_treino":[MSE_teste],
                                             "RMSE_treino":[RMSE_teste],
                                             "MAE_treino":[MAE_teste],
                                             "MAPE_treino":[MAPE_teste]}
}

# Converter para DataFrame
pr_teste_resultados = pd.DataFrame(metricas_teste)
pr_teste_resultados.head()

,Polinomial Regression
r2_teste,[0.09090059850602494]
MSE_treino,[442.6413857657962]
RMSE_treino,[21.039044316836165]
MAE_treino,[16.736414061349993]
MAPE_treino,[827.6971685015494]


#### 2.3.3 Polinomial Regression Lasso

In [13]:
prl_degree = [2, 3, 4]
prl_alpha = [0.1, 1.0, 10.0]
prl_max_iter = [100, 500, 1000]
prl_treino_metricas = {}
prl_val_metricas = {}

for n in prl_degree:
  for m in prl_alpha:
    for o in prl_max_iter:
      #treinamento do modelo variando parâmetros
      prl = PolynomialFeatures(degree=n, include_bias=False)
      prl_features_treino = prl.fit_transform(X_treino)
      prl_model = Lasso(alpha=m, max_iter=o, random_state=13)
      prl_model.fit(prl_features_treino, y_treino_1d)

      #predição de treino
      Y_treino_pred = prl_model.predict(prl_features_treino)

      #métricas de treino
      r2_treino = r2_score(y_treino_1d,Y_treino_pred)
      MSE_treino = mean_squared_error(y_treino_1d,Y_treino_pred)
      RMSE_treino = root_mean_squared_error(y_treino_1d,Y_treino_pred)
      MAE_treino = mean_absolute_error(y_treino_1d,Y_treino_pred)
      MAPE_treino = mean_absolute_percentage_error(y_treino_1d, Y_treino_pred)

      #armazenando na biblioteca
      prl_treino_metricas.update({f"pr_degree{n}, prl_alpha{m} e prl_max_iter{o}": {"r2_treino":r2_treino,
                                                        "MSE_treino":MSE_treino,
                                                        "RMSE_treino":RMSE_treino,
                                                        "MAE_treino":MAE_treino,
                                                        "MAPE_treino":MAPE_treino}})
      #predição de validação
      prl_features_val = prl.transform(X_val)
      Y_val_pred = prl_model.predict(prl_features_val)

      #métricas de treino
      r2_val = r2_score(y_val_1d,Y_val_pred)
      MSE_val = mean_squared_error(y_val_1d,Y_val_pred)
      RMSE_val = root_mean_squared_error(y_val_1d,Y_val_pred)
      MAE_val = mean_absolute_error(y_val_1d,Y_val_pred)
      MAPE_val = mean_absolute_percentage_error(y_val_1d, Y_val_pred)

      #armazenando na biblioteca
      prl_val_metricas.update({f"pr_degree{n}, prl_alpha{m} e prl_max_iter{o}": {"r2_val":r2_val,
                                                      "MSE_val":MSE_val,
                                                      "RMSE_val":RMSE_val,
                                                      "MAE_val":MAE_val,
                                                      "MAPE_val":MAPE_val}})

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.524e+05, tolerance: 5.042e+02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.362e+06, tolerance: 5.042e+02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.334e+06, tolerance: 5.042e

In [14]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(prl_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(prl_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)

                                              r2_treino  MSE_treino  \
pr_degree2, prl_alpha0.1 e prl_max_iter100     0.067936  445.538484   
pr_degree2, prl_alpha0.1 e prl_max_iter500     0.067909  445.551320   
pr_degree2, prl_alpha0.1 e prl_max_iter1000    0.067909  445.551320   
pr_degree2, prl_alpha1.0 e prl_max_iter100     0.009150  473.638776   
pr_degree2, prl_alpha1.0 e prl_max_iter500     0.009150  473.638776   
pr_degree2, prl_alpha1.0 e prl_max_iter1000    0.009150  473.638776   
pr_degree2, prl_alpha10.0 e prl_max_iter100    0.000000  478.012560   
pr_degree2, prl_alpha10.0 e prl_max_iter500    0.000000  478.012560   
pr_degree2, prl_alpha10.0 e prl_max_iter1000   0.000000  478.012560   
pr_degree3, prl_alpha0.1 e prl_max_iter100     0.075692  441.830802   
pr_degree3, prl_alpha0.1 e prl_max_iter500     0.075656  441.848027   
pr_degree3, prl_alpha0.1 e prl_max_iter1000    0.075656  441.848027   
pr_degree3, prl_alpha1.0 e prl_max_iter100     0.014084  471.280382   
pr_deg

**Discussão dos Resultados de Treino e Validação para Métricas de Regressão**

Na seleção do melhor modelo de regressão, o equilíbrio entre performance (capacidade de prever corretamente) e generalização (adaptação a novos dados) é fundamental. As métricas utilizadas (R², MSE, RMSE, MAE e MAPE) fornecem insights complementares sobre o comportamento do modelo, permitindo identificar se ele está sofrendo de overfitting ou underfitting.

**Melhores parâmetros: prl_degree=2, prl_alpha=0.1 e prl_max_iter=100**

Motivos:

Melhor R² na Validação (0.0589), único com R² consistentemente positivo.

Menor MSE na Validação (449.38 vs. 471+ em outras combinações).

MAE mais baixo (16.82 vs. 17+ em alpha=1.0/10.0).

Sem Overfitting: Diferença mínima entre R² treino (0.0679) e validação (0.0589).

In [15]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
prl = PolynomialFeatures(degree=2, include_bias=False)
prl_features_concat = prl.fit_transform(X_concat_treino_val)
prl_model_final = Lasso(alpha=0.1, max_iter=100, random_state=13)
prl_model_final.fit(prl_features_concat, y_concat_treino_val)

#predição de teste
prl_features_teste = prl.transform(X_teste)
Y_teste_pred = prl_model_final.predict(prl_features_teste)

#métricas de teste
r2_teste = r2_score(y_teste_1d,Y_teste_pred)
MSE_teste = mean_squared_error(y_teste_1d,Y_teste_pred)
RMSE_teste = root_mean_squared_error(y_teste_1d,Y_teste_pred)
MAE_teste = mean_absolute_error(y_teste_1d,Y_teste_pred)
MAPE_teste = mean_absolute_percentage_error(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"Polinomial Regression Lasso":{"r2_teste":[r2_teste],
                                             "MSE_treino":[MSE_teste],
                                             "RMSE_treino":[RMSE_teste],
                                             "MAE_treino":[MAE_teste],
                                             "MAPE_treino":[MAPE_teste]}
}

# Converter para DataFrame
prl_teste_resultados = pd.DataFrame(metricas_teste)
prl_teste_resultados.head()

,Polinomial Regression Lasso
r2_teste,[0.06871224983461066]
MSE_treino,[453.4449143872311]
RMSE_treino,[21.294246039417107]
MAE_treino,[16.93249393576484]
MAPE_treino,[847.1998914687784]


#### 2.3.4 Polinomial Regression Ridge

In [18]:
prr_degree = [2, 3, 4]
prr_alpha = [0.1, 1.0, 10.0]
prr_max_iter = [100, 500, 1000]
prr_treino_metricas = {}
prr_val_metricas = {}

for n in prr_degree:
  for m in prr_alpha:
    for o in prr_max_iter:
      #treinamento do modelo variando parâmetros
      prr = PolynomialFeatures(degree=n, include_bias=False)
      prr_features_treino = prr.fit_transform(X_treino)
      prr_model = Ridge(alpha=m, max_iter=o, random_state=13)
      prr_model.fit(prr_features_treino, y_treino_1d)

      #predição de treino
      Y_treino_pred = prr_model.predict(prr_features_treino)

      #métricas de treino
      r2_treino = r2_score(y_treino_1d,Y_treino_pred)
      MSE_treino = mean_squared_error(y_treino_1d,Y_treino_pred)
      RMSE_treino = root_mean_squared_error(y_treino_1d,Y_treino_pred)
      MAE_treino = mean_absolute_error(y_treino_1d,Y_treino_pred)
      MAPE_treino = mean_absolute_percentage_error(y_treino_1d, Y_treino_pred)

      #armazenando na biblioteca
      prr_treino_metricas.update({f"prr_degree{n}, prr_alpha{m} e prr_max_iter{o}": {"r2_treino":r2_treino,
                                                        "MSE_treino":MSE_treino,
                                                        "RMSE_treino":RMSE_treino,
                                                        "MAE_treino":MAE_treino,
                                                        "MAPE_treino":MAPE_treino}})
      #predição de validação
      prr_features_val = prr.transform(X_val)
      Y_val_pred = prr_model.predict(prr_features_val)

      #métricas de treino
      r2_val = r2_score(y_val_1d,Y_val_pred)
      MSE_val = mean_squared_error(y_val_1d,Y_val_pred)
      RMSE_val = root_mean_squared_error(y_val_1d,Y_val_pred)
      MAE_val = mean_absolute_error(y_val_1d,Y_val_pred)
      MAPE_val = mean_absolute_percentage_error(y_val_1d, Y_val_pred)

      #armazenando na biblioteca
      prr_val_metricas.update({f"prr_degree{n}, prr_alpha{m} e prr_max_iter{o}": {"r2_val":r2_val,
                                                      "MSE_val":MSE_val,
                                                      "RMSE_val":RMSE_val,
                                                      "MAE_val":MAE_val,
                                                      "MAPE_val":MAPE_val}})

In [19]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(prr_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(prr_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)

                                               r2_treino  MSE_treino  \
prr_degree2, prr_alpha0.1 e prr_max_iter100     0.094092  433.035331   
prr_degree2, prr_alpha0.1 e prr_max_iter500     0.094092  433.035331   
prr_degree2, prr_alpha0.1 e prr_max_iter1000    0.094092  433.035331   
prr_degree2, prr_alpha1.0 e prr_max_iter100     0.093171  433.475457   
prr_degree2, prr_alpha1.0 e prr_max_iter500     0.093171  433.475457   
prr_degree2, prr_alpha1.0 e prr_max_iter1000    0.093171  433.475457   
prr_degree2, prr_alpha10.0 e prr_max_iter100    0.089350  435.302181   
prr_degree2, prr_alpha10.0 e prr_max_iter500    0.089350  435.302181   
prr_degree2, prr_alpha10.0 e prr_max_iter1000   0.089350  435.302181   
prr_degree3, prr_alpha0.1 e prr_max_iter100     0.147256  407.622157   
prr_degree3, prr_alpha0.1 e prr_max_iter500     0.147256  407.622157   
prr_degree3, prr_alpha0.1 e prr_max_iter1000    0.147256  407.622157   
prr_degree3, prr_alpha1.0 e prr_max_iter100     0.135169  413.40

**Discussão dos Resultados de Treino e Validação para Métricas de Regressão**

Na seleção do melhor modelo de regressão, o equilíbrio entre performance (capacidade de prever corretamente) e generalização (adaptação a novos dados) é fundamental. As métricas utilizadas (R², MSE, RMSE, MAE e MAPE) fornecem insights complementares sobre o comportamento do modelo, permitindo identificar se ele está sofrendo de overfitting ou underfitting.

**Melhores parâmetros: prr_degree=2, prr_alpha=1.0 e prr_max_iter=100**

Motivos:
Melhor R² na Validação (0.0677) entre todas as combinações.

Menor MSE na Validação (445.18 vs. >500 em outros graus polinomiais).

MAE mais baixo (16.74) e consistente com o treino (16.47).

Sem Overfitting:

Diferença mínima entre R² treino (0.093) e validação (0.067).

MSE de validação próximo ao MSE de treino (445.18 vs. 433.48).

Robustez:

Performance estável para max_iter=100/500/1000 (não há ganho com mais iterações).

alpha=1.0 oferece melhor equilíbrio entre viés e variância que alpha=0.1 ou 10.0.

In [20]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
prr = PolynomialFeatures(degree=2, include_bias=False)
prr_features_concat = prr.fit_transform(X_concat_treino_val)
prr_model_final = Ridge(alpha=1.0, max_iter=100, random_state=13)
prr_model_final.fit(prr_features_concat, y_concat_treino_val)

#predição de teste
prr_features_teste = prr.transform(X_teste)
Y_teste_pred = prr_model_final.predict(prr_features_teste)

#métricas de teste
r2_teste = r2_score(y_teste_1d,Y_teste_pred)
MSE_teste = mean_squared_error(y_teste_1d,Y_teste_pred)
RMSE_teste = root_mean_squared_error(y_teste_1d,Y_teste_pred)
MAE_teste = mean_absolute_error(y_teste_1d,Y_teste_pred)
MAPE_teste = mean_absolute_percentage_error(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"Polinomial Regression Ridge":{"r2_teste":[r2_teste],
                                             "MSE_treino":[MSE_teste],
                                             "RMSE_treino":[RMSE_teste],
                                             "MAE_treino":[MAE_teste],
                                             "MAPE_treino":[MAPE_teste]}
}

# Converter para DataFrame
prr_teste_resultados = pd.DataFrame(metricas_teste)
prr_teste_resultados.head()

,Polinomial Regression Ridge
r2_teste,[0.09023105908000861]
MSE_treino,[442.96738516571844]
RMSE_treino,[21.04679037681799]
MAE_treino,[16.742213878468075]
MAPE_treino,[830.850070999773]


#### 2.3.5 Polinomial Regression Elastic Net

In [22]:
pren_degree = [2, 3, 4]
pren_alpha = [0.1, 1.0, 10.0]
pren_l1_ratio = [0.2, 0.5, 0.8]
pren_max_iter = [100, 500, 1000]
pren_treino_metricas = {}
pren_val_metricas = {}

for n in pren_degree:
  for m in pren_alpha:
    for o in pren_l1_ratio:
      for p in prr_max_iter:
        #treinamento do modelo variando parâmetros
        pren = PolynomialFeatures(degree=n, include_bias=False)
        pren_features_treino = pren.fit_transform(X_treino)
        pren_model = ElasticNet(alpha=m, l1_ratio=o, max_iter=p, random_state=13)
        pren_model.fit(pren_features_treino, y_treino_1d)

        #predição de treino
        Y_treino_pred = pren_model.predict(pren_features_treino)

        #métricas de treino
        r2_treino = r2_score(y_treino_1d,Y_treino_pred)
        MSE_treino = mean_squared_error(y_treino_1d,Y_treino_pred)
        RMSE_treino = root_mean_squared_error(y_treino_1d,Y_treino_pred)
        MAE_treino = mean_absolute_error(y_treino_1d,Y_treino_pred)
        MAPE_treino = mean_absolute_percentage_error(y_treino_1d, Y_treino_pred)

        #armazenando na biblioteca
        pren_treino_metricas.update({f"pren_degree{n}, pren_alpha{m}, pren_l1_ratio{o} e pren_max_iter{p}": {"r2_treino":r2_treino,
                                                          "MSE_treino":MSE_treino,
                                                          "RMSE_treino":RMSE_treino,
                                                          "MAE_treino":MAE_treino,
                                                          "MAPE_treino":MAPE_treino}})
        #predição de validação
        pren_features_val = pren.transform(X_val)
        Y_val_pred = pren_model.predict(pren_features_val)

        #métricas de treino
        r2_val = r2_score(y_val_1d,Y_val_pred)
        MSE_val = mean_squared_error(y_val_1d,Y_val_pred)
        RMSE_val = root_mean_squared_error(y_val_1d,Y_val_pred)
        MAE_val = mean_absolute_error(y_val_1d,Y_val_pred)
        MAPE_val = mean_absolute_percentage_error(y_val_1d, Y_val_pred)

        #armazenando na biblioteca
        pren_val_metricas.update({f"pren_degree{n}, pren_alpha{m}, pren_l1_ratio{o} e pren_max_iter{p}": {"r2_val":r2_val,
                                                        "MSE_val":MSE_val,
                                                        "RMSE_val":RMSE_val,
                                                        "MAE_val":MAE_val,
                                                        "MAPE_val":MAPE_val}})

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.165e+06, tolerance: 5.042e+02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.170e+06, tolerance: 5.042e+02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.833e+05, tolerance: 5.042e

In [28]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(pren_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(pren_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)
df_final.to_csv('pren_final.csv')

                                                    r2_treino  MSE_treino  \
pren_degree2, pren_alpha0.1, pren_l1_ratio0.2 e...   0.060223  449.225181   
pren_degree2, pren_alpha0.1, pren_l1_ratio0.2 e...   0.060223  449.225181   
pren_degree2, pren_alpha0.1, pren_l1_ratio0.2 e...   0.060223  449.225181   
pren_degree2, pren_alpha0.1, pren_l1_ratio0.5 e...   0.061135  448.789471   
pren_degree2, pren_alpha0.1, pren_l1_ratio0.5 e...   0.061135  448.789471   
...                                                       ...         ...   
pren_degree4, pren_alpha10.0, pren_l1_ratio0.5 ...   0.001394  477.346095   
pren_degree4, pren_alpha10.0, pren_l1_ratio0.5 ...   0.001394  477.346095   
pren_degree4, pren_alpha10.0, pren_l1_ratio0.8 ...   0.001187  477.445022   
pren_degree4, pren_alpha10.0, pren_l1_ratio0.8 ...   0.001212  477.433435   
pren_degree4, pren_alpha10.0, pren_l1_ratio0.8 ...   0.001212  477.433435   

                                                    RMSE_treino  MAE_treino

**Discussão dos Resultados de Treino e Validação para Métricas de Regressão**

Na seleção do melhor modelo de regressão, o equilíbrio entre performance (capacidade de prever corretamente) e generalização (adaptação a novos dados) é fundamental. As métricas utilizadas (R², MSE, RMSE, MAE e MAPE) fornecem insights complementares sobre o comportamento do modelo, permitindo identificar se ele está sofrendo de overfitting ou underfitting.

**Melhores parâmetros: pren_degree=3, pren_alpha=0.1, pren_l1_ratio=0.8 e pren_max_iter=100**

Motivos:
RMSE_val = 21.242 (o menor entre todas as combinações)

R²_val = 0.055 (o maior entre os modelos com RMSE_val baixo)

MAE_val = 16.829

MAPE_val = 863.769 (ainda alto, mas isso pode ser devido à escala da variável-alvo)

In [29]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
pren = PolynomialFeatures(degree=3, include_bias=False)
pren_features_concat = pren.fit_transform(X_concat_treino_val)
pren_model_final = ElasticNet(alpha=0.1, max_iter=100, random_state=13)
pren_model_final.fit(pren_features_concat, y_concat_treino_val)

#predição de teste
pren_features_teste = pren.transform(X_teste)
Y_teste_pred = pren_model_final.predict(pren_features_teste)

#métricas de teste
r2_teste = r2_score(y_teste_1d,Y_teste_pred)
MSE_teste = mean_squared_error(y_teste_1d,Y_teste_pred)
RMSE_teste = root_mean_squared_error(y_teste_1d,Y_teste_pred)
MAE_teste = mean_absolute_error(y_teste_1d,Y_teste_pred)
MAPE_teste = mean_absolute_percentage_error(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"Polinomial Regression Elastic Net":{"r2_teste":[r2_teste],
                                             "MSE_treino":[MSE_teste],
                                             "RMSE_treino":[RMSE_teste],
                                             "MAE_treino":[MAE_teste],
                                             "MAPE_treino":[MAPE_teste]}
}

# Converter para DataFrame
pren_teste_resultados = pd.DataFrame(metricas_teste)
pren_teste_resultados.head()

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.064e+05, tolerance: 7.200e+02
  model = cd_fast.enet_coordinate_descent(


,Polinomial Regression Elastic Net
r2_teste,[0.0696643113778439]
MSE_treino,[452.98135469272626]
RMSE_treino,[21.28335863280808]
MAE_treino,[16.880615434233007]
MAPE_treino,[847.1319305162738]


#### 2.3.6 Random Forest Regressor

In [30]:
rfr_n_estimators = [100, 500, 1000]
rfr_max_depth = [None, 2, 4, 6, 8, 10]
rfr_treino_metricas = {}
rfr_val_metricas = {}

for m in rfr_n_estimators:
  for n in rfr_max_depth:
      #treinamento do modelo variando parâmetros
      rfr_model = RandomForestRegressor(n_estimators=m, max_depth=n)
      rfr_model.fit(X_treino, y_treino_1d)

      #predição de treino
      Y_treino_pred = rfr_model.predict(X_treino)

      #métricas de treino
      r2_treino = r2_score(y_treino_1d,Y_treino_pred)
      MSE_treino = mean_squared_error(y_treino_1d,Y_treino_pred)
      RMSE_treino = root_mean_squared_error(y_treino_1d,Y_treino_pred)
      MAE_treino = mean_absolute_error(y_treino_1d,Y_treino_pred)
      MAPE_treino = mean_absolute_percentage_error(y_treino_1d, Y_treino_pred)

      #armazenando na biblioteca
      rfr_treino_metricas.update({f"rfr_n_estimators{m} e rfr_max_depth{n}": {"r2_treino":r2_treino,
                                                        "MSE_treino":MSE_treino,
                                                        "RMSE_treino":RMSE_treino,
                                                        "MAE_treino":MAE_treino,
                                                        "MAPE_treino":MAPE_treino}})
      #predição de validação
      Y_val_pred = rfr_model.predict(X_val)

      #métricas de treino
      r2_val = r2_score(y_val_1d,Y_val_pred)
      MSE_val = mean_squared_error(y_val_1d,Y_val_pred)
      RMSE_val = root_mean_squared_error(y_val_1d,Y_val_pred)
      MAE_val = mean_absolute_error(y_val_1d,Y_val_pred)
      MAPE_val = mean_absolute_percentage_error(y_val_1d, Y_val_pred)

      #armazenando na biblioteca
      rfr_val_metricas.update({f"rfr_n_estimators{m} e rfr_max_depth{n}": {"r2_val":r2_val,
                                                      "MSE_val":MSE_val,
                                                      "RMSE_val":RMSE_val,
                                                      "MAE_val":MAE_val,
                                                      "MAPE_val":MAPE_val}})

In [31]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(rfr_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(rfr_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)

                                          r2_treino  MSE_treino  RMSE_treino  \
rfr_n_estimators100 e rfr_max_depthNone    0.903022   46.356486     6.808560   
rfr_n_estimators100 e rfr_max_depth2       0.051037  453.616251    21.298269   
rfr_n_estimators100 e rfr_max_depth4       0.106806  426.957962    20.662961   
rfr_n_estimators100 e rfr_max_depth6       0.187471  388.398990    19.707841   
rfr_n_estimators100 e rfr_max_depth8       0.311656  329.036907    18.139374   
rfr_n_estimators100 e rfr_max_depth10      0.471366  252.693896    15.896349   
rfr_n_estimators500 e rfr_max_depthNone    0.906305   44.787440     6.692342   
rfr_n_estimators500 e rfr_max_depth2       0.050347  453.945922    21.306007   
rfr_n_estimators500 e rfr_max_depth4       0.107135  426.800633    20.659154   
rfr_n_estimators500 e rfr_max_depth6       0.187967  388.161896    19.701825   
rfr_n_estimators500 e rfr_max_depth8       0.312800  328.490277    18.124301   
rfr_n_estimators500 e rfr_max_depth10   

**Discussão dos Resultados de Treino e Validação para Métricas de Regressão**

Na seleção do melhor modelo de regressão, o equilíbrio entre performance (capacidade de prever corretamente) e generalização (adaptação a novos dados) é fundamental. As métricas utilizadas (R², MSE, RMSE, MAE e MAPE) fornecem insights complementares sobre o comportamento do modelo, permitindo identificar se ele está sofrendo de overfitting ou underfitting.

**Melhores parâmetros: n_estimators=1000 e max_depth=None**

Motivos:
RMSE_val = 17.731 (o menor entre todas as combinações)

R²_val = 0.3416 (o mais alto, indicando melhor explicação da variância)

MAE_val = 12.928 (menor erro absoluto médio)

MAPE_val = 704.115 (ainda alto, mas isso pode ser devido à escala da variável-alvo)

In [32]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
rfr_model_final = RandomForestRegressor(n_estimators=1000, max_depth=None)
rfr_model_final.fit(X_concat_treino_val, y_concat_treino_val)

#predição de teste
Y_teste_pred = rfr_model_final.predict(X_teste)

#métricas de teste
r2_teste = r2_score(y_teste_1d,Y_teste_pred)
MSE_teste = mean_squared_error(y_teste_1d,Y_teste_pred)
RMSE_teste = root_mean_squared_error(y_teste_1d,Y_teste_pred)
MAE_teste = mean_absolute_error(y_teste_1d,Y_teste_pred)
MAPE_teste = mean_absolute_percentage_error(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"Random Forest Regressor":{"r2_teste":[r2_teste],
                                             "MSE_treino":[MSE_teste],
                                             "RMSE_treino":[RMSE_teste],
                                             "MAE_treino":[MAE_teste],
                                             "MAPE_treino":[MAPE_teste]}
}

# Converter para DataFrame
rfr_teste_resultados = pd.DataFrame(metricas_teste)
rfr_teste_resultados.head()

,Random Forest Regressor
r2_teste,[0.4084441890522196]
MSE_treino,[288.0291016421599]
RMSE_treino,[16.971420142173134]
MAE_treino,[12.17460422289223]
MAPE_treino,[633.340742289936]


#### 2.3.7 Linear Regression Lasso

In [33]:
lrl_alpha = [0.1, 1.0, 10.0]
lrl_max_iter = [100, 500, 1000]
lrl_treino_metricas = {}
lrl_val_metricas = {}

for m in lrl_alpha:
  for n in lrl_max_iter:
      #treinamento do modelo variando parâmetros
      lrl_model = Lasso(alpha=m, max_iter=n, random_state=13)
      lrl_model.fit(X_treino, y_treino_1d)

      #predição de treino
      Y_treino_pred = lrl_model.predict(X_treino)

      #métricas de treino
      r2_treino = r2_score(y_treino_1d,Y_treino_pred)
      MSE_treino = mean_squared_error(y_treino_1d,Y_treino_pred)
      RMSE_treino = root_mean_squared_error(y_treino_1d,Y_treino_pred)
      MAE_treino = mean_absolute_error(y_treino_1d,Y_treino_pred)
      MAPE_treino = mean_absolute_percentage_error(y_treino_1d, Y_treino_pred)

      #armazenando na biblioteca
      lrl_treino_metricas.update({f"lrl_alpha{m} e lrl_max_iter{n}": {"r2_treino":r2_treino,
                                                        "MSE_treino":MSE_treino,
                                                        "RMSE_treino":RMSE_treino,
                                                        "MAE_treino":MAE_treino,
                                                        "MAPE_treino":MAPE_treino}})
      #predição de validação
      Y_val_pred = lrl_model.predict(X_val)

      #métricas de treino
      r2_val = r2_score(y_val_1d,Y_val_pred)
      MSE_val = mean_squared_error(y_val_1d,Y_val_pred)
      RMSE_val = root_mean_squared_error(y_val_1d,Y_val_pred)
      MAE_val = mean_absolute_error(y_val_1d,Y_val_pred)
      MAPE_val = mean_absolute_percentage_error(y_val_1d, Y_val_pred)

      #armazenando na biblioteca
      lrl_val_metricas.update({f"lrl_alpha{m} e lrl_max_iter{n}": {"r2_val":r2_val,
                                                      "MSE_val":MSE_val,
                                                      "RMSE_val":RMSE_val,
                                                      "MAE_val":MAE_val,
                                                      "MAPE_val":MAPE_val}})

In [34]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(lrl_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(lrl_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)

                                  r2_treino  MSE_treino  RMSE_treino  \
lrl_alpha0.1 e lrl_max_iter100     0.041219  458.309397    21.408162   
lrl_alpha0.1 e lrl_max_iter500     0.041219  458.309397    21.408162   
lrl_alpha0.1 e lrl_max_iter1000    0.041219  458.309397    21.408162   
lrl_alpha1.0 e lrl_max_iter100     0.007401  474.474834    21.782443   
lrl_alpha1.0 e lrl_max_iter500     0.007401  474.474834    21.782443   
lrl_alpha1.0 e lrl_max_iter1000    0.007401  474.474834    21.782443   
lrl_alpha10.0 e lrl_max_iter100    0.000000  478.012560    21.863498   
lrl_alpha10.0 e lrl_max_iter500    0.000000  478.012560    21.863498   
lrl_alpha10.0 e lrl_max_iter1000   0.000000  478.012560    21.863498   

                                  MAE_treino  MAPE_treino        r2_val  \
lrl_alpha0.1 e lrl_max_iter100     17.046776   866.796391  3.719533e-02   
lrl_alpha0.1 e lrl_max_iter500     17.046776   866.796391  3.719533e-02   
lrl_alpha0.1 e lrl_max_iter1000    17.046776   866.796

**Discussão dos Resultados de Treino e Validação para Métricas de Regressão**

Na seleção do melhor modelo de regressão, o equilíbrio entre performance (capacidade de prever corretamente) e generalização (adaptação a novos dados) é fundamental. As métricas utilizadas (R², MSE, RMSE, MAE e MAPE) fornecem insights complementares sobre o comportamento do modelo, permitindo identificar se ele está sofrendo de overfitting ou underfitting.

**Melhores parâmetros: lrl_alpha=0.1 e lrl_max_iter=100**

Motivos:
RMSE_val = 21.441 (o menor entre todas as combinações)

R²_val = 0.0372 (o mais alto, indicando melhor explicação da variância)

MAE_val = 17.047 (menor erro absoluto médio no validação)

MAPE_val = 868.690 (ainda alto, mas consistente com outros modelos)

In [35]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
lrl_model_final = Lasso(alpha=0.1, max_iter=100, random_state=13)
lrl_model_final.fit(X_concat_treino_val, y_concat_treino_val)

#predição de teste
Y_teste_pred = lrl_model_final.predict(X_teste)

#métricas de teste
r2_teste = r2_score(y_teste_1d,Y_teste_pred)
MSE_teste = mean_squared_error(y_teste_1d,Y_teste_pred)
RMSE_teste = root_mean_squared_error(y_teste_1d,Y_teste_pred)
MAE_teste = mean_absolute_error(y_teste_1d,Y_teste_pred)
MAPE_teste = mean_absolute_percentage_error(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"Linear Regression Lasso":{"r2_teste":[r2_teste],
                                             "MSE_treino":[MSE_teste],
                                             "RMSE_treino":[RMSE_teste],
                                             "MAE_treino":[MAE_teste],
                                             "MAPE_treino":[MAPE_teste]}
}

# Converter para DataFrame
lrl_teste_resultados = pd.DataFrame(metricas_teste)
lrl_teste_resultados.head()

,Linear Regression Lasso
r2_teste,[0.042959617738577704]
MSE_treino,[465.983896086451]
RMSE_treino,[21.58666014200555]
MAE_treino,[17.194746532197033]
MAPE_treino,[860.2586628881018]


#### 2.3.8 Linear Regression Ridge

In [38]:
lrr_alpha = [0.1, 1.0, 10.0]
lrr_max_iter = [100, 500, 1000]
lrr_treino_metricas = {}
lrr_val_metricas = {}

for m in lrr_alpha:
  for n in lrr_max_iter:
      #treinamento do modelo variando parâmetros
      lrr_model = Ridge(alpha=m, max_iter=n, random_state=13)
      lrr_model.fit(X_treino, y_treino_1d)

      #predição de treino
      Y_treino_pred = lrr_model.predict(X_treino)

      #métricas de treino
      r2_treino = r2_score(y_treino_1d,Y_treino_pred)
      MSE_treino = mean_squared_error(y_treino_1d,Y_treino_pred)
      RMSE_treino = root_mean_squared_error(y_treino_1d,Y_treino_pred)
      MAE_treino = mean_absolute_error(y_treino_1d,Y_treino_pred)
      MAPE_treino = mean_absolute_percentage_error(y_treino_1d, Y_treino_pred)

      #armazenando na biblioteca
      lrr_treino_metricas.update({f"lrr_alpha{m} e lrr_max_iter{n}": {"r2_treino":r2_treino,
                                                        "MSE_treino":MSE_treino,
                                                        "RMSE_treino":RMSE_treino,
                                                        "MAE_treino":MAE_treino,
                                                        "MAPE_treino":MAPE_treino}})
      #predição de validação
      Y_val_pred = lrr_model.predict(X_val)

      #métricas de treino
      r2_val = r2_score(y_val_1d,Y_val_pred)
      MSE_val = mean_squared_error(y_val_1d,Y_val_pred)
      RMSE_val = root_mean_squared_error(y_val_1d,Y_val_pred)
      MAE_val = mean_absolute_error(y_val_1d,Y_val_pred)
      MAPE_val = mean_absolute_percentage_error(y_val_1d, Y_val_pred)

      #armazenando na biblioteca
      lrr_val_metricas.update({f"lrr_alpha{m} e lrr_max_iter{n}": {"r2_val":r2_val,
                                                      "MSE_val":MSE_val,
                                                      "RMSE_val":RMSE_val,
                                                      "MAE_val":MAE_val,
                                                      "MAPE_val":MAPE_val}})

In [39]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(lrr_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(lrr_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)

                                  r2_treino  MSE_treino  RMSE_treino  \
lrr_alpha0.1 e lrr_max_iter100     0.046058  455.996115    21.354066   
lrr_alpha0.1 e lrr_max_iter500     0.046058  455.996115    21.354066   
lrr_alpha0.1 e lrr_max_iter1000    0.046058  455.996115    21.354066   
lrr_alpha1.0 e lrr_max_iter100     0.046058  455.996401    21.354072   
lrr_alpha1.0 e lrr_max_iter500     0.046058  455.996401    21.354072   
lrr_alpha1.0 e lrr_max_iter1000    0.046058  455.996401    21.354072   
lrr_alpha10.0 e lrr_max_iter100    0.046009  456.019763    21.354619   
lrr_alpha10.0 e lrr_max_iter500    0.046009  456.019763    21.354619   
lrr_alpha10.0 e lrr_max_iter1000   0.046009  456.019763    21.354619   

                                  MAE_treino  MAPE_treino    r2_val  \
lrr_alpha0.1 e lrr_max_iter100     16.998255   865.320909  0.039925   
lrr_alpha0.1 e lrr_max_iter500     16.998255   865.320909  0.039925   
lrr_alpha0.1 e lrr_max_iter1000    16.998255   865.320909  0.03992

**Discussão dos Resultados de Treino e Validação para Métricas de Regressão**

Na seleção do melhor modelo de regressão, o equilíbrio entre performance (capacidade de prever corretamente) e generalização (adaptação a novos dados) é fundamental. As métricas utilizadas (R², MSE, RMSE, MAE e MAPE) fornecem insights complementares sobre o comportamento do modelo, permitindo identificar se ele está sofrendo de overfitting ou underfitting.

**Melhores parâmetros: lrr_alpha=10.0 e lrr_max_iter=100**

Motivos:

Apresenta o menor RMSE_val (21.411238) entre todas as configurações

Maior R²_val (0.039937) - melhor capacidade explicativa

Menor MAE_val (17.037610)

Menor MAPE_val (868.134247)

Todas as métricas de validação são marginalmente melhores que outras configurações

Diferenças pequenas, mas consistentes em todas as métricas

alpha=10.0 mostrou ligeira vantagem sobre alpha=1.0 e alpha=0.1

Indica que um maior nível de regularização foi benéfico para este conjunto de dados

max_iter=100 é suficiente (resultados idênticos para 500 e 1000 iterações)

Não há ganho com mais iterações

In [40]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
lrr_model_final = Ridge(alpha=10.0, max_iter=100, random_state=13)
lrr_model_final.fit(X_concat_treino_val, y_concat_treino_val)

#predição de teste
Y_teste_pred = lrr_model_final.predict(X_teste)

#métricas de teste
r2_teste = r2_score(y_teste_1d,Y_teste_pred)
MSE_teste = mean_squared_error(y_teste_1d,Y_teste_pred)
RMSE_teste = root_mean_squared_error(y_teste_1d,Y_teste_pred)
MAE_teste = mean_absolute_error(y_teste_1d,Y_teste_pred)
MAPE_teste = mean_absolute_percentage_error(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"Linear Regression Ridge":{"r2_teste":[r2_teste],
                                             "MSE_treino":[MSE_teste],
                                             "RMSE_treino":[RMSE_teste],
                                             "MAE_treino":[MAE_teste],
                                             "MAPE_treino":[MAPE_teste]}
}

# Converter para DataFrame
lrr_teste_resultados = pd.DataFrame(metricas_teste)
lrr_teste_resultados.head()

,Linear Regression Ridge
r2_teste,[0.05112843600186201]
MSE_treino,[462.0064905022135]
RMSE_treino,[21.494336242420083]
MAE_treino,[17.142430127057033]
MAPE_treino,[853.7815914841639]


#### 2.3.8 Linear Regression Elastic Net

In [42]:
lren_alpha = [0.1, 1.0, 10.0]
lren_l1_ratio = [0.2, 0.5, 0.8]
lren_max_iter = [100, 500, 1000]
lren_treino_metricas = {}
lren_val_metricas = {}


for m in lren_alpha:
  for n in lren_l1_ratio:
    for o in lren_max_iter:
        #treinamento do modelo variando parâmetros
        lren_model = ElasticNet(alpha=m, l1_ratio=n, max_iter=o, random_state=13)
        lren_model.fit(X_treino, y_treino_1d)

        #predição de treino
        Y_treino_pred = lren_model.predict(X_treino)

        #métricas de treino
        r2_treino = r2_score(y_treino_1d,Y_treino_pred)
        MSE_treino = mean_squared_error(y_treino_1d,Y_treino_pred)
        RMSE_treino = root_mean_squared_error(y_treino_1d,Y_treino_pred)
        MAE_treino = mean_absolute_error(y_treino_1d,Y_treino_pred)
        MAPE_treino = mean_absolute_percentage_error(y_treino_1d, Y_treino_pred)

        #armazenando na biblioteca
        lren_treino_metricas.update({f"lren_alpha{m}, lren_l1_ratio{n} e lren_max_iter{o}": {"r2_treino":r2_treino,
                                                          "MSE_treino":MSE_treino,
                                                          "RMSE_treino":RMSE_treino,
                                                          "MAE_treino":MAE_treino,
                                                          "MAPE_treino":MAPE_treino}})
        #predição de validação
        Y_val_pred = lren_model.predict(X_val)

        #métricas de treino
        r2_val = r2_score(y_val_1d,Y_val_pred)
        MSE_val = mean_squared_error(y_val_1d,Y_val_pred)
        RMSE_val = root_mean_squared_error(y_val_1d,Y_val_pred)
        MAE_val = mean_absolute_error(y_val_1d,Y_val_pred)
        MAPE_val = mean_absolute_percentage_error(y_val_1d, Y_val_pred)

        #armazenando na biblioteca
        lren_val_metricas.update({f"lren_alpha{m}, lren_l1_ratio{n} e lren_max_iter{o}": {"r2_val":r2_val,
                                                        "MSE_val":MSE_val,
                                                        "RMSE_val":RMSE_val,
                                                        "MAE_val":MAE_val,
                                                        "MAPE_val":MAPE_val}})

In [43]:
# Converter cada dicionário em DataFrame
df_treino = pd.DataFrame(lren_treino_metricas).T  # .T para transpor (chaves viram linhas)
df_val = pd.DataFrame(lren_val_metricas).T

# Combinar horizontalmente (concatenação por colunas)
df_final = pd.concat([df_treino, df_val], axis=1)

print(df_final)

                                                    r2_treino  MSE_treino  \
lren_alpha0.1, lren_l1_ratio0.2 e lren_max_iter100   0.027771  464.737447   
lren_alpha0.1, lren_l1_ratio0.2 e lren_max_iter500   0.027771  464.737447   
lren_alpha0.1, lren_l1_ratio0.2 e lren_max_iter...   0.027771  464.737447   
lren_alpha0.1, lren_l1_ratio0.5 e lren_max_iter100   0.030495  463.435615   
lren_alpha0.1, lren_l1_ratio0.5 e lren_max_iter500   0.030495  463.435615   
lren_alpha0.1, lren_l1_ratio0.5 e lren_max_iter...   0.030495  463.435615   
lren_alpha0.1, lren_l1_ratio0.8 e lren_max_iter100   0.035440  461.071843   
lren_alpha0.1, lren_l1_ratio0.8 e lren_max_iter500   0.035440  461.071843   
lren_alpha0.1, lren_l1_ratio0.8 e lren_max_iter...   0.035440  461.071843   
lren_alpha1.0, lren_l1_ratio0.2 e lren_max_iter100   0.009403  473.517863   
lren_alpha1.0, lren_l1_ratio0.2 e lren_max_iter500   0.009403  473.517863   
lren_alpha1.0, lren_l1_ratio0.2 e lren_max_iter...   0.009403  473.517863   

**Discussão dos Resultados de Treino e Validação para Métricas de Regressão**

Na seleção do melhor modelo de regressão, o equilíbrio entre performance (capacidade de prever corretamente) e generalização (adaptação a novos dados) é fundamental. As métricas utilizadas (R², MSE, RMSE, MAE e MAPE) fornecem insights complementares sobre o comportamento do modelo, permitindo identificar se ele está sofrendo de overfitting ou underfitting.

**Melhores parâmetros: lren_alpha=0.1, lren_l1_ratio=0.8 e lren_max_iter=100**

Motivos:

Menor RMSE_val (21.4937) entre todas as configurações

Maior R²_val (0.03252) - melhor capacidade explicativa

MAE_val mais baixo (17.0676)

MAPE_val (867.885) consistente com outras configurações

Configurações com alpha=0.1 superam claramente alpha=1.0 e alpha=10.0

l1_ratio=0.8 (mais ênfase em L1) mostrou melhor desempenho que 0.2 e 0.5

Número de iterações não afeta resultados (100 já é suficiente)

alpha=0.1: Fornece regularização suficiente sem causar underfitting

l1_ratio=0.8: Combinação ideal entre seleção de features (L1) e regularização ridge (L2)

max_iter=100: Convergência alcançada sem necessidade de mais iterações

In [45]:
#Treinamento do modelo com os melhores parâmetros encontrados e com dataset de treino e validação concatenados
lren_model_final = ElasticNet(alpha=0.1, l1_ratio=0.8, max_iter=100, random_state=13)
lren_model_final.fit(X_concat_treino_val, y_concat_treino_val)

#predição de teste
Y_teste_pred = lren_model_final.predict(X_teste)

#métricas de teste
r2_teste = r2_score(y_teste_1d,Y_teste_pred)
MSE_teste = mean_squared_error(y_teste_1d,Y_teste_pred)
RMSE_teste = root_mean_squared_error(y_teste_1d,Y_teste_pred)
MAE_teste = mean_absolute_error(y_teste_1d,Y_teste_pred)
MAPE_teste = mean_absolute_percentage_error(y_teste_1d,Y_teste_pred)

# Criar um dicionário com as métricas
metricas_teste = {"Linear Regression Elastic Net":{"r2_teste":[r2_teste],
                                             "MSE_treino":[MSE_teste],
                                             "RMSE_treino":[RMSE_teste],
                                             "MAE_treino":[MAE_teste],
                                             "MAPE_treino":[MAPE_teste]}
}

# Converter para DataFrame
lren_teste_resultados = pd.DataFrame(metricas_teste)
lren_teste_resultados.head()

,Linear Regression Elastic Net
r2_teste,[0.03728582505947686]
MSE_treino,[468.7464712788863]
RMSE_treino,[21.650553602134202]
MAE_treino,[17.22441085719846]
MAPE_treino,[864.4994352895244]


### 2.4 Resultados e Discussão

In [46]:
# Função para reformatar cada DataFrame
def formatar_dataframe(df, nome_modelo):
    # Extrai os valores das listas e cria um novo DataFrame
    df_reformatado = pd.DataFrame({
        'Métrica': df.index,
        'Valor': df.iloc[:, 0].str[0],  # Extrai o valor dentro da lista
        'Modelo': nome_modelo
    })
    return df_reformatado

# Lista de DataFrames originais e seus nomes
dataframes_originais = [dtr_teste_resultados, pr_teste_resultados, prl_teste_resultados, prr_teste_resultados, pren_teste_resultados, rfr_teste_resultados, lrl_teste_resultados, lrr_teste_resultados, lren_teste_resultados]
nomes_modelos = ['Decision Tree Regressor', 'Polinomial Regression', 'Polinomial Regression Lasso', 'Polinomial Regression Ridge', 'Polinomial Regression Elastic Net', 'Random Forest Regressor', 'Linear Regression Lasso', 'Linear Regression Ridge', 'Linear Regression Elastic Net']

# Reformata e concatena todos
resultados_finais = pd.concat(
    [formatar_dataframe(df, nome) for df, nome in zip(dataframes_originais, nomes_modelos)],
    ignore_index=True
)

# Opcional: Converter para formato wide (métricas como colunas)
resultados_pivot = resultados_finais.pivot(
    index='Modelo',
    columns='Métrica',
    values='Valor'
).reset_index()

print(resultados_pivot)

Métrica                             Modelo  MAE_treino  MAPE_treino  \
0                  Decision Tree Regressor   16.679876   772.830453   
1            Linear Regression Elastic Net   17.224411   864.499435   
2                  Linear Regression Lasso   17.194747   860.258663   
3                  Linear Regression Ridge   17.142430   853.781591   
4                    Polinomial Regression   16.736414   827.697169   
5        Polinomial Regression Elastic Net   16.880615   847.131931   
6              Polinomial Regression Lasso   16.932494   847.199891   
7              Polinomial Regression Ridge   16.742214   830.850071   
8                  Random Forest Regressor   12.174604   633.340742   

Métrica  MSE_treino  RMSE_treino  r2_teste  
0        437.376433    20.913547  0.101714  
1        468.746471    21.650554  0.037286  
2        465.983896    21.586660  0.042960  
3        462.006491    21.494336  0.051128  
4        442.641386    21.039044  0.090901  
5        452.981355

**Melhor Modelo: Random Forest Regressor**

Menor MAE_treino (12.17) - Erro absoluto médio significativamente menor que outros modelos.

Menor MAPE_treino (633.34) - Apesar de ainda alto, é o mais baixo entre todos.

Menor MSE_treino (288.03) e RMSE_treino (16.97) - Indicam melhor precisão nas previsões.

Maior R²_teste (0.408) - Explica ~40% da variância nos dados de teste, o melhor entre os modelos.

## 3.0 Ensaio de Agrupamento

### 3.1 Importando e lendo o dataset

In [2]:
X_agrupamento = pd.read_csv('Ensaio_de_Clusterizacao/X_dataset.csv')

### 3.2 Explorando o dataset

In [3]:
X_agrupamento.head()

,alcohol,malic_acid,ash,ash_alcanity,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280,proline
0,1.518613,0.191700,0.232053,-1.169593,1.913905,0.627586,0.573840,-0.659563,1.224884,0.251717,0.455285,0.970696,0.561341
1,0.246290,0.205534,-0.827996,-2.490847,0.018145,0.575862,0.510549,-0.820719,-0.544721,-0.293321,0.463415,0.780220,0.550642
2,0.196879,0.320158,1.109334,-0.268738,0.088358,0.627586,0.611814,-0.498407,2.135968,0.269020,0.447154,0.695971,0.646933
3,1.691550,0.239130,0.487926,-0.809251,0.930918,0.989655,0.664557,-0.981875,1.032155,1.186068,0.308943,0.798535,0.857347
4,0.295700,0.365613,1.840403,0.451946,1.281985,0.627586,0.495781,0.226796,0.401404,-0.319276,0.455285,0.608059,0.325963


In [4]:
X_agrupamento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   alcohol               178 non-null    float64
 1   malic_acid            178 non-null    float64
 2   ash                   178 non-null    float64
 3   ash_alcanity          178 non-null    float64
 4   magnesium             178 non-null    float64
 5   total_phenols         178 non-null    float64
 6   flavanoids            178 non-null    float64
 7   nonflavanoid_phenols  178 non-null    float64
 8   proanthocyanins       178 non-null    float64
 9   color_intensity       178 non-null    float64
 10  hue                   178 non-null    float64
 11  od280                 178 non-null    float64
 12  proline               178 non-null    float64
dtypes: float64(13)
memory usage: 18.2 KB


**Comentários**

Há um total de 178 entradas (linhas).

Com 13 colunas (features), todas correspondendo a valores numéricos (tipo float64) e que já foram normalizados.

Não há dados faltantes nem erros de digitação.

### 3.3 Análises

#### 3.3.1 K-Means

In [11]:
km_n_clusters = [3, 5, 8, 11]
km_silhouette_scores = {}

for n in km_n_clusters:
  # Aplicar K-means
  km_model = KMeans(n_clusters=n, random_state=13, n_init='auto')
  km_labels = km_model.fit_predict(X_agrupamento)

  # Calcular silhouette score
  silhouette_avg = silhouette_score(X_agrupamento, km_labels)
  km_silhouette_scores[n] = silhouette_avg

km_silhouette_scores_df = pd.DataFrame.from_dict(
    km_silhouette_scores,
    orient='index',
    columns=['Silhouette Score']
)
km_silhouette_scores_df.index.name = 'N Clusters'
km_silhouette_scores_df.columns.name = 'K-means'  # Nomeia a coluna como K-means

display(km_silhouette_scores_df)

K-means,Silhouette Score
N Clusters,
3,0.232959
5,0.216430
8,0.178245
11,0.175864


O Silhouette Score mede a qualidade dos clusters, variando de -1 a 1, onde:

Valores próximos de 1: Indica que os clusters estão bem separados e os pontos estão próximos de seus próprios clusters.

Valores próximos de 0: Indica que os clusters estão muito próximos ou se sobrepõem.

Valores negativos: Indica que muitos pontos foram atribuídos ao cluster errado.

**Melhor parâmetro: km_n_clusters=3**

Teve o maior Silhouette Score (0.233), indicando que essa configuração produz clusters mais bem definidos.

À medida que aumentamos o número de clusters, o score diminui, sugerindo que:

- Mais clusters podem estar dividindo grupos naturais desnecessariamente.

- Pode haver overfitting, onde clusters adicionais não trazem melhorias significativas na estrutura dos dados.

#### 3.3.2 Affinity Propagation

In [19]:
ap_preferences = np.linspace(-200, 20, 50)
ap_silhouette_scores = {}

for n in ap_preferences:
  # Aplicar K-means
  ap_model = AffinityPropagation(preference=n, random_state=13)
  ap_labels = ap_model.fit_predict(X_agrupamento)

  # Calcular silhouette score
  n_clusters = len(np.unique(ap_labels))
  if 1 < n_clusters < len(X_agrupamento):
        silhouette_avg = silhouette_score(X_agrupamento, ap_labels)
        ap_silhouette_scores[n] = (silhouette_avg, n_clusters)
  else:
        ap_silhouette_scores[n] = (np.nan, n_clusters)

# Create DataFrame
ap_results_df = pd.DataFrame.from_dict(
    ap_silhouette_scores,
    orient='index',
    columns=['Silhouette Score', 'Number of Clusters']
)
ap_results_df.index.name = 'Preference'
display(ap_results_df.sort_values(by='Silhouette Score', ascending=False))

,Silhouette Score,Number of Clusters
Preference,,
-132.653061,0.223820,3
-155.102041,0.215123,3
-146.122449,0.215123,3
-141.632653,0.215123,3
-182.040816,0.215123,3
-47.346939,0.203658,7
-51.836735,0.202286,7
-56.326531,0.201180,7
-173.061224,0.198860,3


O Silhouette Score mede a qualidade dos clusters, variando de -1 a 1, onde:

Valores próximos de 1: Indica que os clusters estão bem separados e os pontos estão próximos de seus próprios clusters.

Valores próximos de 0: Indica que os clusters estão muito próximos ou se sobrepõem.

Valores negativos: Indica que muitos pontos foram atribuídos ao cluster errado.

**Melhor parâmetro: ap_preferences=-132.653061**
preference = -132.653061 (3 clusters, Silhouette 0.2238) é a melhor escolha porque:

Tem o maior Silhouette Score (clusters mais bem definidos).

3 clusters é um número interpretável (não fragmenta demais os dados).

Evita overfitting, ao contrário de configurações com dezenas de clusters.

### 3.4 Resultados e Discussão

Desempenho Geral
K-means obteve um Silhouette Score ligeiramente superior (0.2329 vs. 0.2238) na configuração de 3 clusters. Ambos os métodos indicam que 3 clusters é uma estrutura natural nos dados, já que produzem os melhores scores nessa configuração.

**Vantagens do K-means**
Mais consistente em controlar o número de clusters

Enquanto o Affinity Propagation varia muito o número de clusters conforme o preference, o K-means permite definir diretamente n_clusters, facilitando a comparação.

Melhor resultado absoluto (0.2329 > 0.2238)

Apesar da diferença pequena, o K-means agrupou os dados com clusters ligeiramente mais bem definidos.

**Vantagens do Affinity Propagation**
Não requer definição prévia do número de clusters

Descobriu automaticamente que 3 clusters era uma boa solução (sem precisar testar valores manualmente). Mas é sensível ao parâmetro preference o que pode gerar muitos clusters ruins (ex: 75 ou até 178 clusters com scores baixíssimos).

**Conclusão: Qual Método é Melhor?**
K-means (n_clusters=3) é a melhor escolha, pois atingiu o maior Silhouette Score (0.2329).

Se o objetivo é descobrir automaticamente o número de clusters, Affinity Propagation pode ser útil, mas requer ajuste cuidadoso do preference.